# Plots for verification report to Hydro Tasmania, July 2018

In [ ]:
%cd /OSM/CBR/OA_DCFP/work/squ027/doppyo/
! python setup.py install >/dev/null

In [ ]:
from doppyo import utils, skill, diagnostic

import glob
import itertools
import numpy as np
import xarray as xr
import pandas as pd
import dask.bag as db

import cartopy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# Notebook specific -----
%matplotlib inline

In [ ]:
import dask
import distributed
client = distributed.Client('tcp://oa-32-cdc.nexus.csiro.au:8786')
client

In [ ]:
# Reload pyLatte -----
import importlib
utils = importlib.reload(utils)
diagnostic = importlib.reload(diagnostic)
skill = importlib.reload(skill)

In [ ]:
# Save location -----
saveloc = '/OSM/CBR/OA_DCFP/work/squ027/squire_scratch/writing/reports/hydro_verif_2018/figures/'

In [ ]:
# Define sequential colormap -----
colors = [(0.0315,    0.1138,    0.3452),
    (0.0483,    0.1196,    0.3737),
    (0.0645,    0.1258,    0.4015),
    (0.0791,    0.1329,    0.4277),
    (0.0926,    0.1399,    0.4528),
    (0.1053,    0.1468,    0.4772),
    (0.1170,    0.1570,    0.5003),
    (0.1278,    0.1672,    0.5223),
    (0.1351,    0.1760,    0.5398),
    (0.1403,    0.1854,    0.5560),
    (0.1433,    0.1956,    0.5708),
    (0.1447,    0.2069,    0.5816),
    (0.1457,    0.2189,    0.5913),
    (0.1456,    0.2331,    0.6001),
    (0.1448,    0.2476,    0.6085),
    (0.1429,    0.2622,    0.6165),
    (0.1414,    0.2779,    0.6230),
    (0.1402,    0.2940,    0.6289),
    (0.1396,    0.3096,    0.6349),
    (0.1386,    0.3257,    0.6409),
    (0.1367,    0.3432,    0.6472),
    (0.1356,    0.3608,    0.6550),
    (0.1351,    0.3786,    0.6639),
    (0.1345,    0.3964,    0.6728),
    (0.1337,    0.4146,    0.6821),
    (0.1324,    0.4346,    0.6933),
    (0.1309,    0.4541,    0.7045),
    (0.1291,    0.4727,    0.7160),
    (0.1285,    0.4931,    0.7257),
    (0.1282,    0.5141,    0.7348),
    (0.1285,    0.5335,    0.7440),
    (0.1294,    0.5527,    0.7516),
    (0.1314,    0.5711,    0.7564),
    (0.1373,    0.5897,    0.7605),
    (0.1454,    0.6083,    0.7643),
    (0.1617,    0.6261,    0.7683),
    (0.1796,    0.6438,    0.7720),
    (0.2008,    0.6619,    0.7746),
    (0.2243,    0.6794,    0.7780),
    (0.2499,    0.6964,    0.7822),
    (0.2753,    0.7127,    0.7850),
    (0.3009,    0.7287,    0.7875),
    (0.3284,    0.7432,    0.7907),
    (0.3568,    0.7571,    0.7935),
    (0.3861,    0.7703,    0.7958),
    (0.4187,    0.7818,    0.7981),
    (0.4525,    0.7927,    0.8004),
    (0.4876,    0.8059,    0.8026),
    (0.5229,    0.8188,    0.8058),
    (0.5586,    0.8312,    0.8111),
    (0.5951,    0.8439,    0.8163),
    (0.6321,    0.8569,    0.8216),
    (0.6672,    0.8701,    0.8293),
    (0.7020,    0.8833,    0.8375),
    (0.7363,    0.8965,    0.8467),
    (0.7693,    0.9094,    0.8562),
    (0.8012,    0.9223,    0.8661),
    (0.8302,    0.9338,    0.8761),
    (0.8580,    0.9447,    0.8862),
    (0.8815,    0.9532,    0.8962),
    (0.9028,    0.9613,    0.9060),
    (0.9207,    0.9686,    0.9156),
    (0.9565,    0.9828,    0.9521),
    (1.0000,    1.0000,    1.0000),
    (1.0000,    0.9999,    0.9711),
    (1.0000,    0.9998,    0.9422),
    (1.0000,    0.9996,    0.9134),
    (1.0000,    0.9995,    0.8845),
    (1.0000,    0.9994,    0.8556),
    (1.0000,    0.9993,    0.8267),
    (1.0000,    0.9991,    0.7979),
    (1.0000,    0.9990,    0.7690),
    (1.0000,    0.9989,    0.7401),
    (1.0000,    0.9966,    0.7216),
    (1.0000,    0.9907,    0.7068),
    (1.0000,    0.9836,    0.6928),
    (1.0000,    0.9744,    0.6767),
    (1.0000,    0.9643,    0.6614),
    (1.0000,    0.9528,    0.6476),
    (0.9984,    0.9397,    0.6322),
    (0.9961,    0.9259,    0.6161),
    (0.9961,    0.9120,    0.6022),
    (0.9961,    0.8982,    0.5878),
    (0.9961,    0.8844,    0.5717),
    (0.9961,    0.8706,    0.5542),
    (0.9961,    0.8568,    0.5358),
    (0.9961,    0.8390,    0.5174),
    (0.9961,    0.8205,    0.4990),
    (0.9961,    0.8021,    0.4805),
    (0.9950,    0.7826,    0.4611),
    (0.9927,    0.7619,    0.4403),
    (0.9922,    0.7394,    0.4231),
    (0.9922,    0.7165,    0.4070),
    (0.9922,    0.6958,    0.3908),
    (0.9913,    0.6743,    0.3755),
    (0.9890,    0.6512,    0.3617),
    (0.9867,    0.6267,    0.3494),
    (0.9844,    0.6014,    0.3379),
    (0.9821,    0.5760,    0.3264),
    (0.9793,    0.5501,    0.3148),
    (0.9747,    0.5225,    0.3033),
    (0.9701,    0.4961,    0.2918),
    (0.9655,    0.4708,    0.2803),
    (0.9608,    0.4474,    0.2707),
    (0.9559,    0.4246,    0.2612),
    (0.9490,    0.4039,    0.2497),
    (0.9421,    0.3822,    0.2382),
    (0.9352,    0.3592,    0.2266),
    (0.9283,    0.3395,    0.2151),
    (0.9213,    0.3210,    0.2036),
    (0.9097,    0.3003,    0.1921),
    (0.8990,    0.2803,    0.1813),
    (0.8898,    0.2619,    0.1721),
    (0.8777,    0.2420,    0.1643),
    (0.8639,    0.2213,    0.1574),
    (0.8522,    0.2027,    0.1547),
    (0.8401,    0.1843,    0.1529),
    (0.8263,    0.1658,    0.1529),
    (0.8113,    0.1474,    0.1518),
    (0.7952,    0.1290,    0.1495),
    (0.7772,    0.1106,    0.1490),
    (0.7585,    0.0921,    0.1490),
    (0.7378,    0.0737,    0.1490),
    (0.7162,    0.0553,    0.1490),
    (0.6931,    0.0369,    0.1490),
    (0.6701,    0.0184,    0.1490),
    (0.6471,         0,    0.1490)]

cmap_name = 'squire_div'
seq = matplotlib.colors.LinearSegmentedColormap.from_list(cmap_name, colors, N=len(colors))
seq.set_bad('w',1.)

In [ ]:
cmap_lin = matplotlib.cm.get_cmap('YlGnBu_r')
levels = 1 - np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11,12,13,14,15,16,17,18,19,20,25,30,35, 40,45, 50,55, 60,65, 70, 80, 90, 95, 99]) / 100
cmap_nonlin = cmap_lin(np.flip(levels,0))
cmap_name = 'nonlin_div'
div_nonlin = matplotlib.colors.LinearSegmentedColormap.from_list(cmap_name, cmap_nonlin, N=len(cmap_nonlin))
div_nonlin.set_bad('w',1.)

In [ ]:
cmap_name = 'div'
div = matplotlib.colors.LinearSegmentedColormap.from_list(cmap_name, cmap_lin(np.linspace(0,1,100)), N=len(cmap_lin(np.linspace(0,1,100))))
div.set_bad('w',1.)

# Functions

In [ ]:
def below_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') < tercile_bnd)


def above_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') > tercile_bnd)


def sample_normal_dist(da,mn,sd):
    da_out = da.copy()
    months = da.time.dt.month.values
    da_out.values = np.random.normal(mn.sel(month=months), sd.sel(month=months))
    return da_out


def populate_chance(da,mn,sd):
    da_out = utils.leadtime_to_datetime(da).groupby('ensemble').apply(sample_normal_dist, mn=mn, sd=sd)
    return utils.datetime_to_leadtime(da_out)

# Global preciptation metrics

### Load required data

In [ ]:
# Forecast data -----
folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'cafe.fcst_2yr.v1.precip.bundle.monthly.native_grid.20030101_20151201.new.nc'

precip_f = xr.open_dataset(folder + file)['precip'] * 60 * 60 * 24 / 998.2 * 1000

# Observation data -----
folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'JRA55.precip.daily.cafe_grid.20020101_201612331.nc'

precip_o = utils.stack_by_init_date(xr.open_dataset(folder + file)['precip'] \
                                      .resample(time='MS').sum('time'),
                                    precip_f['init_date'].values, 24)

### Compute the terciles for each month

In [ ]:
# Compute the terciles for each month from the forecasts -----
dat_list = []
for idx in range(len(precip_f.init_date)):
    dat_list.append(utils.leadtime_to_datetime(precip_f.isel(init_date=idx)))
precip_fs = xr.concat(dat_list, dim='time')

t12_f = precip_fs.groupby('time.month').reduce(np.nanpercentile, dim=['time', 'ensemble'], q=100/3)
t23_f = precip_fs.groupby('time.month').reduce(np.nanpercentile, dim=['time', 'ensemble'], q=200/3)

# Compute the terciles for each month from the observations -----
dat_list = []
for idx in range(len(precip_o.init_date)):
    dat_list.append(utils.leadtime_to_datetime(precip_o.isel(init_date=idx)))
precip_os = xr.concat(dat_list, dim='time')

t12_o = precip_os.groupby('time.month').reduce(np.nanpercentile, dim=['time'], q=100/3)
t23_o = precip_os.groupby('time.month').reduce(np.nanpercentile, dim=['time'], q=200/3)

### Compute the Brier score relative to chance

#### Compute Brier score for forecasts

In [ ]:
precip_f_l1 = precip_f.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f).mean(dim='ensemble')
precip_o_l1 = precip_o.groupby('init_date').apply(below_tercile, tercile_bnd=t12_o)

precip_f_l3 = precip_f.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f).mean(dim='ensemble')
precip_o_l3 = precip_o.groupby('init_date').apply(above_tercile, tercile_bnd=t23_o)

# Compute the Brier score for forecasts -----
Brier_precip_f_t1 = skill.compute_Brier_score(precip_f_l1, precip_o_l1, over_dims=['init_date'])
Brier_precip_f_t3 = skill.compute_Brier_score(precip_f_l3, precip_o_l3, over_dims=['init_date'])

#### Compute the Brier score for chance

In [ ]:
# Mean and std of normal distribution used to sample chance -----
mn = precip_fs.groupby('time.month').mean('time')
sd = precip_fs.groupby('time.month').std('time')

chance = precip_f.copy().groupby('init_date').apply(populate_chance, mn=mn, sd=sd)

In [ ]:
precip_x_l1 = chance.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f).mean(dim='ensemble')
precip_x_l3 = chance.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f).mean(dim='ensemble')
    
Brier_precip_x_t1 = skill.compute_Brier_score(precip_x_l1, precip_o_l1, over_dims=['init_date'])
Brier_precip_x_t3 = skill.compute_Brier_score(precip_x_l3, precip_o_l3, over_dims=['init_date'])

#### Compute relative skill

In [ ]:
Brier_precip_fx_t1 = 1 - (Brier_precip_f_t1 / Brier_precip_x_t1)
Brier_precip_fx_t3 = 1 - (Brier_precip_f_t3 / Brier_precip_x_t3)

### Compute the Brier score relative to climatology

#### Compute the Brier score for climatology

In [ ]:
precip_c_l1 = (0 * precip_f_l1 + 1/3)
precip_c_l3 = (0 * precip_f_l3 + 1/3)
Brier_precip_c_t1 = skill.compute_Brier_score(precip_c_l1, precip_o_l1, over_dims=['init_date'])
Brier_precip_c_t3 = skill.compute_Brier_score(precip_c_l3, precip_o_l3, over_dims=['init_date'])

#### Compute relative skill

In [ ]:
Brier_precip_fc_t1 = 1 - (Brier_precip_f_t1 / Brier_precip_c_t1)
Brier_precip_fc_t3 = 1 - (Brier_precip_f_t3 / Brier_precip_c_t3)

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_fx_t1.isel(lead_time=3), Brier_precip_fx_t3.isel(lead_time=3), 
        Brier_precip_fc_t1.isel(lead_time=3), Brier_precip_fc_t3.isel(lead_time=3)]
text = ['(a)','(b)','(c)','(d)']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 19})

ncol = 2; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3.7))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='grey')
    ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    # im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=-1, vmax=1, cmap=seq)
    im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,21), origin='lower', transform=trans, 
                      vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_ylim(extent[2],extent[3])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    if idx / ncol < nrow - 1:
        gl.xlabels_bottom = False
    gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.text(-170, -80, text[idx])
    
    count += 1
        
plt.tight_layout()
fig.subplots_adjust(bottom=0.19)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.024])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS / BS_{\mathrm{ref}}$', rotation=0, labelpad=15)
cbar.set_ticks(np.linspace(-1,1,11))
plt.savefig(saveloc + 'brier_skill_chanceclim_rainfall_month4_global.eps', dpi=600, format='eps', bbox_inches='tight')

# Global monthly SST metrics

In [ ]:
folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'cafe.fcst_2yr.v1.sst.bundle.monthly.native_grid.20030101_20151201.nc'

sst_f = xr.open_dataset(folder + file)['sst']

folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'REMSS.sst.monthly.cafe_grid.20020601_20171201.nc'

sst_o_raw = xr.open_dataset(folder + file)['sst'] - 273.15

### Only keep the parts of both datasets where neither are nans

In [ ]:
# Function to fill common nans -----
def common_nans(da_1, da_2, return_da=1):
    """ Returns da_1 or da_2 with nans where either da_1 or da_2 contain nans """

    # Find overlapping time -----
    tmin = xr.concat([da_1['time'].min(),da_2['time'].min()], dim='time').max()
    tmax = xr.concat([da_1['time'].max(),da_2['time'].max()], dim='time').min()
    tmin_s = f'{tmin.dt.year.values}-{tmin.dt.month.values}-{tmin.dt.day.values}'
    tmax_s = f'{tmax.dt.year.values}-{tmax.dt.month.values}-{tmax.dt.day.values}'
    
    da1 = da_1.sel(time=slice(tmin_s,tmax_s))
    da2 = da_2.sel(time=slice(tmin_s,tmax_s))

    keep = da1.notnull() & da2.notnull()

    if return_da == 1:
        daout = da1.where(keep)
    elif return_da == 2:
        daout = da2.where(keep)

    return daout

In [ ]:
# Fill common nans -----
both_nans = lambda da1, da2, ret: utils.datetime_to_leadtime(
                                     common_nans(
                                         utils.leadtime_to_datetime(da1),da2,ret))

sst_f = sst_f.groupby('init_date').apply(both_nans, da2=sst_o_raw, ret=1)
    
sst_o = sst_f.groupby('init_date').apply(both_nans, da2=sst_o_raw, ret=2) \
                     .isel(ensemble=[0],drop=True).squeeze()

### Mask the sea ice

In [ ]:
# Load mask -----
folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'REMSS.sst.monthly.cafe_grid.20020601_20171201.nc'
ice_mask = xr.open_dataset(folder + file)['sea_ice_fraction'] > 0
ice_mask = ice_mask.where(ice_mask == 0)

In [ ]:
# Convert sea ice to nans -----
sst_f = sst_f.groupby('init_date').apply(both_nans, da2=ice_mask, ret=1)
    
sst_o = sst_o.groupby('init_date').apply(both_nans, da2=ice_mask, ret=1)

### Load and compute the sst anomalies

In [ ]:
# Load forecast climatology -----
sst_f_clim = utils.load_mean_climatology('cafe_f1_ocean_2003-2017', freq='MS', variable='sst') \
                  .rename({'lat_t':'lat','lon_t':'lon'}).compute()

# Load observation climatology -----
sst_o_clim = utils.load_mean_climatology('REMSS_2002-2018',  freq='MS', variable='sst').compute() - 273.15

In [ ]:
anomalize = lambda data, clim: utils.datetime_to_leadtime(
                                   utils.anomalize(
                                       utils.leadtime_to_datetime(data),clim))

sst_f_anom = sst_f.groupby('init_date').apply(anomalize, clim=sst_f_clim)

sst_o_anom = sst_o.groupby('init_date').apply(anomalize, clim=sst_o_clim)

sst_o_raw_anom = utils.anomalize(sst_o_raw, clim=sst_o_clim)

### Compute the anomaly correlation

In [ ]:
anom_corr = skill.compute_Pearson_corrcoef(sst_f_anom.mean(dim='ensemble'), sst_o_anom, 
                                           over_dims=['init_date'], subtract_local_mean=True)

In [ ]:
vmin = -0.9
vmax = 0.9

data = [anom_corr.isel(lead_time=3), anom_corr.isel(lead_time=7), 
        anom_corr.isel(lead_time=11), anom_corr.isel(lead_time=15)]
text = ['(a)','(b)','(c)','(d)']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 19})

ncol = 2; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3.7))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='grey')
    ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    # im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=-1, vmax=1, cmap=seq)
    im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,21), origin='lower', transform=trans, 
                      vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_ylim(extent[2],extent[3])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    if idx / ncol < nrow - 1:
        gl.xlabels_bottom = False
    gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.text(-170,-70,text[idx])
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.19)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.024])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('SST anomaly correlation', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.8,0.8,9))
plt.savefig(saveloc + 'anomaly_correlation_sst.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.9
vmax = 0.9

data = [anom_corr.isel(lead_time=3), anom_corr.isel(lead_time=7), 
        anom_corr.isel(lead_time=11), anom_corr.isel(lead_time=15)]
text = ['4 months lead','8 months lead','12 months lead','16 months lead']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 16})

ncol = 2; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3.7))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='grey')
    ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    # im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=-1, vmax=1, cmap=seq)
    im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,21), origin='lower', transform=trans, 
                      vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_ylim(extent[2],extent[3])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    if idx / ncol < nrow - 1:
        gl.xlabels_bottom = False
    gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.set_title(text[idx],fontsize=16)
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.19)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.024])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('SST anomaly correlation', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.8,0.8,9))
plt.savefig(saveloc + 'anomaly_correlation_sst_forpres.eps', dpi=600, format='eps', bbox_inches='tight')

# Monthly Nino3.4 metrics

### Compute nino3.4 index

In [ ]:
nino34_f = diagnostic.compute_nino34(sst_f_anom)
nino34_o = diagnostic.compute_nino34(sst_o_anom)
nino34_o_raw = diagnostic.compute_nino34(sst_o_raw_anom)

### Split init_date into init_month and init_year

In [ ]:
def date_to_monthyear(da):
    """ Split init_date dimension into init_month and init_year dimensions """
    
    da['init_month'] = da.init_date.dt.month
    da['init_year'] = da.init_date.dt.year

    return da.set_index(init_date=['init_year', 'init_month']).unstack('init_date')

In [ ]:
nino34_f = date_to_monthyear(nino34_f)
nino34_o = date_to_monthyear(nino34_o)

### Compute the anolmaly correlation

In [ ]:
year_to_include = 13

anom_corr = skill.compute_Pearson_corrcoef(nino34_f.mean(dim='ensemble').isel(init_year=range(year_to_include)), 
                                           nino34_o.isel(init_year=range(year_to_include)), 
                                           over_dims=['init_year'], subtract_local_mean=True)

In [ ]:
fig = plt.figure(figsize=(10, 10))

im = plt.imshow(anom_corr.transpose(), origin='lower', vmin=-1, vmax=1, cmap=seq)
plt.xlabel('Lead months')
plt.yticks(np.linspace(0,11,12),['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.ylabel('Start month')

plt.tight_layout()
fig.subplots_adjust(bottom=0.01)
cbar_ax = fig.add_axes([0.15, 0.18, 0.7, 0.015])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Nino3.4 anomaly correlation coefficient', rotation=0, labelpad=15);

# Daily Nino3.4 metrics

### Load required data

#### Load forecast data

In [ ]:
# For some reason, I cannot use pyLatte functions within functions of dask.bag -----
def calc_boxavg_latlon(da, box):
    '''
        Returns the average of a given quantity over a provide lat-lon region
    '''
    
    return da.sel(lat=slice(box[0],box[1]), lon=slice(box[2],box[3])).mean(dim=['lat', 'lon'])

def compute_nino34(da_sst_anom):
    ''' Returns nino3.4 index '''  
    
    box = [-5.0,5.0,360.0-170.0,360.0-120.0] # [lat_min,lat_max,lon_min,lon_max]
    
    # Account for datasets with negative longitudes -----
    if np.any(da_sst_anom['lon'] < 0):
        box[2] = box[2] - 360
        box[3] = box[3] - 360
        
    return calc_boxavg_latlon(da_sst_anom,box)

In [ ]:
def open_clip_and_expand(path, variable, new_coords, clip_time, chunks):
    """ Returns DataArray with added coordinates as specified by dictionary: new_coords """

    # Path may glob multiple files - use open_mfdataset -----
    da = compute_nino34(xr.open_mfdataset(path, autoclose=True, parallel=True)[variable] \
                          .chunk(chunks=chunks) \
                          .rename({'xt_ocean' : 'lon', 'yt_ocean' : 'lat'}))
    
    if (clip_time is not None):
        da = da.isel(time=range(min([clip_time,len(da.time)])))

    # Add new coordinates -----
    for key, value in zip(new_coords.keys(), new_coords.values()):
        if key not in da.dims:
            da.coords[key] = value
    
    return da


def load_ncfile(path, variable, freq, convert_to_leadtime, clip_time, chunks):
    """ 
        Load specified variable from specified path
        
        convert_to_leadtime = True converts the 'time' dimension to a 'lead_time' dimension
    """
    
    da = open_clip_and_expand(path[-1], variable, path[0], clip_time, chunks)

    # Convert "time" dimension to a "time since date" (lead_time) dimension -----
    if convert_to_leadtime:
        da['time'] =  np.arange(len(da[convert_to_leadtime]))
        da = da.rename({'time' : 'lead_time'})
        da.lead_time.attrs = {'units': freq}

    return da


def load_and_concat(paths, variable, freq, chunks=None, convert_to_leadtime=False, clip_time=None):
    """
        Load specified files in parallel and concatenate into single array
        
        Parallel loading takes approximately 0.75 sec per file with 16 workers
        Serial loading takes approximately 5.25 sec per file with 16 workers
    """
    
    # Initialise dask bag -----
    bag = db.from_sequence(paths, partition_size=1)
    da_list = bag.map(load_ncfile, variable=variable, freq=freq, convert_to_leadtime=convert_to_leadtime, 
                      clip_time=clip_time, chunks=chunks).compute()
    
    return xr.concat(da_list, dim='stack').set_index(stack=list(paths[0][0].keys())).unstack('stack')

In [ ]:
# Initial dates to include -----
init_dates = pd.date_range('2002-6','2015-12' , freq='1MS')

# Ensembles to include -----
ensembles = range(1,12)

# Generate list of paths and coordinates to create - to be replaced by the database approach when working again -----
filepath = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1'
filename = 'ocean_daily*.nc'
paths = [({'init_date' : i, 'ensemble' : e}, filepath + '/yr' + str(i.year) + '/mn' + str(i.month) + \
                '/OUTPUT.' + str(e) + '/' + filename) 
         for i,e in itertools.product(init_dates, ensembles)]

In [ ]:
%%time

anomalize = lambda data, clim: utils.datetime_to_leadtime(
                                   utils.anomalize(
                                       utils.leadtime_to_datetime(data),clim))

# Load forecast raw -----
nino34_f_raw = load_and_concat(paths, variable='sst', freq='D', chunks={'time' : 731}, 
                               convert_to_leadtime='time', clip_time=731).compute()

# Load forecast climatology -----
sst_f_clim = utils.load_mean_climatology('cafe_f1_ocean_2003-2017', 'sst', freq='D') \
                  .rename({'lat_t':'lat','lon_t':'lon'}).compute()
nino34_f_clim = compute_nino34(sst_f_clim)

# Compute nino3.4 -----
nino34_f = nino34_f_raw.groupby('init_date').apply(anomalize, clim=nino34_f_clim)

#### Load observation data

In [ ]:
def load_remss_data(path, variable):
    return compute_nino34(xr.open_mfdataset(path, parallel=True)[variable]).chunk(chunks={'time':100})

In [ ]:
# Initial dates to include -----
years = np.unique(init_dates.year)

# Generate list of paths and coordinates to create - to be replaced by the database approach when working again -----
filepath = '/OSM/CBR/OA_DCFP/data/observations/sst/remss/v05.0'
filename = '*REMSS*.nc'
paths = [filepath + '/' + str(year) + '/' + filename for year in years]

In [ ]:
%%time
bag = db.from_sequence(paths, partition_size=1)
da_list = bag.map(load_remss_data, variable='analysed_sst').compute()

nino34_o_raw = xr.concat(da_list, dim='time').compute() - 273.15
nino34_o_raw['time'] = nino34_o_raw['time'].astype('datetime64[D]')

sst_o_clim = utils.load_mean_climatology('REMSS_2002-2018', 'sst', freq='D').compute() - 273.15
nino34_o_clim = compute_nino34(sst_o_clim)

nino34_o = utils.stack_by_init_date(utils.anomalize(nino34_o_raw,nino34_f_clim), init_dates, 731)

#### Build persistence

In [ ]:
nino34_p = utils.repeat_data(nino34_o,'lead_time')

### Split init_date into init_month and init_year

In [ ]:
nino34_f_split = date_to_monthyear(nino34_f)
nino34_o_split = date_to_monthyear(nino34_o)
nino34_p_split = date_to_monthyear(nino34_p)

### Compute the anolmaly correlation

#### For the forecasts

In [ ]:
year_to_include = 14
anom_corr_f = skill.compute_Pearson_corrcoef(nino34_f_split.mean(dim='ensemble').isel(init_year=range(year_to_include)), 
                                           nino34_o_split.isel(init_year=range(year_to_include)), 
                                           over_dims=['init_year'], subtract_local_mean=True)

In [ ]:
fig = plt.figure(figsize=(8,6))

data = anom_corr_f.transpose()

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 16})

#im = plt.imshow(data, origin='lower', vmin=0.5, vmax=0.9, cmap='Reds', aspect=35)
im = plt.contourf(data['lead_time'], data['init_month'], data, np.linspace(-1,1,21),
                  origin='lower', vmin=-1, vmax=1, cmap=seq, extend='both');
plt.contour(data['lead_time'], data['init_month'], data, np.linspace(-1,1,11), 
                  linewidths=1, origin='lower',colors='black')
plt.xlabel('Lead days')
plt.yticks(np.linspace(1,12,12),['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.ylabel('Start month')

plt.tight_layout()
fig.subplots_adjust(bottom=0.325)
cbar_ax = fig.add_axes([0.18, 0.18, 0.7, 0.025])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Nino3.4 anomaly correlation coefficient', rotation=0, labelpad=15);

plt.savefig(saveloc + 'nino34_anomcorr.eps', dpi=600, format='eps', bbox_inches='tight')

#### For persistence

In [ ]:
year_to_include = 14
anom_corr_p = skill.compute_Pearson_corrcoef(nino34_p_split.isel(init_year=range(year_to_include)), 
                                             nino34_o_split.isel(init_year=range(year_to_include)), 
                                             over_dims=['init_year'], subtract_local_mean=True)

In [ ]:
fig = plt.figure(figsize=(10,6))

data = anom_corr_p.transpose()

#im = plt.imshow(data, origin='lower', vmin=0.5, vmax=0.9, cmap='Reds', aspect=35)
im = plt.contourf(data['lead_time'], data['init_month'], data, np.linspace(-1,1,41),
                  origin='lower', vmin=-1, vmax=1, cmap=seq)
plt.contour(data['lead_time'], data['init_month'], data, np.linspace(-1,1,11), 
                  origin='lower',colors='black')
plt.xlabel('Lead days')
plt.yticks(np.linspace(1,12,12),['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.ylabel('Start month')

plt.tight_layout()
fig.subplots_adjust(bottom=0.3)
cbar_ax = fig.add_axes([0.18, 0.18, 0.7, 0.018])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Nino3.4 anomaly correlation coefficient', rotation=0, labelpad=15);

#### Difference between forecasts and persistence

In [ ]:
fig = plt.figure(figsize=(10,6))

data = (anom_corr_f - anom_corr_p).transpose()

#im = plt.imshow(data, origin='lower', vmin=0.5, vmax=0.9, cmap='Reds', aspect=35)
im = plt.contourf(data['lead_time'], data['init_month'], data, np.linspace(-1,1,41),
                  origin='lower', vmin=-1, vmax=1, cmap=seq)
plt.contour(data['lead_time'], data['init_month'], data, np.linspace(-1,1,11), 
                  origin='lower',colors='black')
plt.xlabel('Lead days')
plt.yticks(np.linspace(1,12,12),['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.ylabel('Start month')

plt.tight_layout()
fig.subplots_adjust(bottom=0.3)
cbar_ax = fig.add_axes([0.18, 0.18, 0.7, 0.018])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Nino3.4 anomaly correlation coefficient', rotation=0, labelpad=15);

# Local precipitation metrics

In [ ]:
folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'cafe.fcst_2yr.v1.precip.bundle.daily.aus.native_grid.20030101_20151201.nc'

precip_f_d = xr.open_dataset(folder + file)['precip'] * 60 * 60 * 24 / 998.2 * 1000
precip_f_d['init_date'] = precip_f_d.init_date.astype('<M8[D]')                  

In [ ]:
folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'JRA55.precip.daily.cafe_grid.20020101_201612331.nc'

precip_o_raw = xr.open_dataset(folder + file)['precip']
precip_o_raw['time'] = precip_o_raw.time.astype('<M8[D]')
precip_o_d = utils.stack_by_init_date(precip_o_raw, precip_f_d.init_date.values, 2*365) \
                  .sel(lat=slice(-47,-10), lon=slice(110,155))

In [ ]:
landsea = xr.open_dataset('/OSM/CBR/OA_DCFP/data/intermediate_products/masks/ncar_landsea_mask.nc')['LSMASK']
mask = landsea.interp(lat=precip_f_d.lat, lon=precip_f_d.lon, method='nearest')

In [ ]:
# Compute monthly rainfall -----
convert_to_monthly = lambda data : utils.datetime_to_leadtime(
                                   utils.leadtime_to_datetime(data).resample(time='MS').sum(dim='time',skipna=False))

precip_f_m = precip_f_d.groupby('init_date').apply(convert_to_monthly).where(mask)
precip_o_m = precip_o_d.groupby('init_date').apply(convert_to_monthly).where(mask)

### Climatologies

In [ ]:
# Forecasts -----
precip_fc_d = utils.load_mean_climatology('cafe_f1_atmos_2003-2017', variable='precip', freq='D') \
                   .sel(lat=slice(-47,-10), lon=slice(110,155)).where(mask) * 60 * 60 * 24 / 998.2 * 1000
precip_fc_m = utils.load_mean_climatology('cafe_f1_atmos_2003-2017', variable='precip', freq='MS') \
                    .sel(lat=slice(-47,-10), lon=slice(110,155)).where(mask) * 60 * 60 * 24 / 998.2 * 1000

# Observations -----
precip_oc_d = utils.load_mean_climatology('jra_1958-2016', variable='precip', freq='D') \
                    .interp(lat=precip_f_d.lat, lon=precip_f_d.lon, method='linear').where(mask)
precip_oc_m = utils.load_mean_climatology('jra_1958-2016', variable='precip', freq='MS') \
                    .interp(lat=precip_f_m.lat, lon=precip_f_m.lon, method='linear').where(mask)

### Plot seasonal averages

In [ ]:
precip_f_seas = precip_fc_d.groupby('time.season').mean('time')
precip_o_seas = precip_oc_d.groupby('time.season').mean('time')

In [ ]:
vmin = 0
vmax = 10

data = [precip_f_seas.isel(season=0), precip_f_seas.isel(season=2), 
        precip_f_seas.isel(season=1), precip_f_seas.isel(season=3),
        precip_o_seas.isel(season=0), precip_o_seas.isel(season=2), 
        precip_o_seas.isel(season=1), precip_o_seas.isel(season=3)]
text = ['(a)','(b)','(c)','(d)','(e)','(f)','(g)','(h)','(i)','(j)','(k)','(l)']
text2 = ['DJF','MAM','JJA','SON','DJF','MAM','JJA','SON']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180), facecolor='w')
    ax.coastlines(color='black')
    ax.background_patch.set_facecolor('w')
    ax.add_feature(cartopy.feature.LAND, facecolor='w')
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]

    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=div_nonlin)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    ax.text(-67.5, -14, text2[idx], fontsize=14)
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Seasonal daily mean rainfall [mm]', rotation=0, labelpad=15);

plt.savefig(saveloc + 'seasonal_mean_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -6
vmax = 6

data = [(precip_f_seas-precip_o_seas).isel(season=0), (precip_f_seas-precip_o_seas).isel(season=2), 
        (precip_f_seas-precip_o_seas).isel(season=1), (precip_f_seas-precip_o_seas).isel(season=3)]
text = ['(a)','(b)','(c)','(d)']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*4.0))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.01)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.042])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Difference in seasonal daily mean rainfall [mm]', rotation=0, labelpad=15);
# plt.savefig(saveloc + 'difference_seasonal_mean_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Compute tersiles edges from historical data

In [ ]:
def cftime_to_datetime64(time,shift_year=0):
    """ 
    Convert cftime object to datetime64 object
    
    Assumes times are sequential
    """

    if (time.values[0].timetuple()[0]+shift_year < 1678) | (time.values[-1].timetuple()[0]+shift_year > 2261):
        raise ValueError('Cannot create datetime64 object for years outside on 1678-2262')
        
    return np.array([np.datetime64(time.values[i].replace(year=time.values[i].timetuple()[0]+shift_year) \
                                                 .strftime(), 'ns') \
                                                 for i in range(len(time))])

#### Control run

In [ ]:
folder = '/OSM/CBR/OA_DCFP/data2/model_output/CAFE/controls/c2/OUTPUT/'
file = 'atmos_daily_04*.nc'

precip_fh_d = xr.open_mfdataset(folder + file, concat_dim='time', parallel=True)['precip'] \
                .sel(lat=slice(-47,-10), lon=slice(110,155)) \
                .where(mask).compute() * 60 * 60 * 24 / 998.2 * 1000
precip_fh_d['time'] = cftime_to_datetime64(precip_fh_d['time'],1300)
precip_fh_d = precip_fh_d.squeeze()

# Keep only last 40 years -----
precip_fh_d = precip_fh_d.sel(time=slice('1760-01-01','1799-12-31'))

precip_fh_m = precip_fh_d.resample(time='MS').sum('time')

t12_fh_precip = precip_fh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=100/3)
t23_fh_precip = precip_fh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=200/3)

#### JRA

In [ ]:
folder = '/OSM/CBR/OA_DCFP/data/observations/jra55/isobaric/061_tprat/cat/'
file = 'jra.55.tprat.000.1958010100_2016123121.nc'

precip_oh_d = xr.open_dataset(folder + file)['TPRAT_GDS0_SFC_ave3h'] \
                .sel(initial_time0_hours=slice('1958-01-01','1997-12-31')) \
                .mean('forecast_time1') \
                .rename({'g0_lon_3':'lon','g0_lat_2':'lat',
                     'initial_time0_hours':'time'}) \
                .interp(lat=precip_fh_d.lat, lon=precip_fh_d.lon, method='linear') \
                .where(mask)
precip_oh_d = precip_oh_d.squeeze()

precip_oh_m = precip_oh_d.resample(time='MS').sum('time')

t12_oh_precip = precip_oh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=100/3)
t23_oh_precip = precip_oh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=200/3)

### Probability anomalies

In [ ]:
def below_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') < tercile_bnd)

def above_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') > tercile_bnd)

#### Tersile forecast probability anomalies

In [ ]:
precip_f_l3 = precip_f_m.groupby('init_date').apply(above_tercile, tercile_bnd=t23_fh_precip).mean(dim='ensemble')
precip_f_l1 = precip_f_m.groupby('init_date').apply(below_tercile, tercile_bnd=t12_fh_precip).mean(dim='ensemble')

probanom_f_t3 = precip_f_l3.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1 = precip_f_l1.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3.isel(lead_time=0), probanom_f_t3.isel(lead_time=3), 
        probanom_f_t3.isel(lead_time=7), probanom_f_t3.isel(lead_time=11),
        probanom_f_t1.isel(lead_time=1), probanom_f_t1.isel(lead_time=4),
        probanom_f_t1.isel(lead_time=7), probanom_f_t1.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Forecast probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

#### Tersile observation probability anomalies

In [ ]:
precip_o_l3 = precip_o_m.groupby('init_date').apply(above_tercile, tercile_bnd=t23_oh_precip)
precip_o_l1 = precip_o_m.groupby('init_date').apply(below_tercile, tercile_bnd=t12_oh_precip)

probanom_o_t3 = precip_o_l3.where(mask).mean(dim='init_date') - 1/3
probanom_o_t1 = precip_o_l1.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_o_t3.isel(lead_time=0), probanom_o_t3.isel(lead_time=3), 
        probanom_o_t3.isel(lead_time=7), probanom_o_t3.isel(lead_time=11),
        probanom_o_t1.isel(lead_time=1), probanom_o_t1.isel(lead_time=4),
        probanom_o_t1.isel(lead_time=7), probanom_o_t1.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Observation probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Clearly the distributions have drifted (particularly the forecasts), so Brier skill is poor

#### Compute Brier score for forecasts

In [ ]:
Brier_precip_f_t3 = skill.compute_Brier_score(precip_f_l3, precip_o_l3, over_dims=['init_date'])
Brier_precip_f_t1 = skill.compute_Brier_score(precip_f_l1, precip_o_l1, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_f_t1.isel(lead_time=0), Brier_precip_f_t1.isel(lead_time=3), 
        Brier_precip_f_t1.isel(lead_time=7), Brier_precip_f_t1.isel(lead_time=11),
        Brier_precip_f_t3.isel(lead_time=0), Brier_precip_f_t3.isel(lead_time=3),
        Brier_precip_f_t3.isel(lead_time=7), Brier_precip_f_t3.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
precip_c_l3 = (0 * precip_f_l3 + 1/3).where(mask)
precip_c_l1 = (0 * precip_f_l1 + 1/3).where(mask)

Brier_precip_c_t3 = skill.compute_Brier_score(precip_c_l3, precip_o_l3, over_dims=['init_date'])
Brier_precip_c_t1 = skill.compute_Brier_score(precip_c_l1, precip_o_l1, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_c_t1.isel(lead_time=0), Brier_precip_c_t1.isel(lead_time=3), 
        Brier_precip_c_t1.isel(lead_time=7), Brier_precip_c_t1.isel(lead_time=11),
        Brier_precip_c_t3.isel(lead_time=0), Brier_precip_c_t3.isel(lead_time=3),
        Brier_precip_c_t3.isel(lead_time=7), Brier_precip_c_t3.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_precip_fc_t3 = 1 - (Brier_precip_f_t3 / Brier_precip_c_t3).where(mask)
Brier_precip_fc_t1 = 1 - (Brier_precip_f_t1 / Brier_precip_c_t1).where(mask)

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_fc_t1.isel(lead_time=0), Brier_precip_fc_t1.isel(lead_time=3), 
        Brier_precip_fc_t1.isel(lead_time=7), Brier_precip_fc_t1.isel(lead_time=11),
        Brier_precip_fc_t3.isel(lead_time=0), Brier_precip_fc_t3.isel(lead_time=3),
        Brier_precip_fc_t3.isel(lead_time=7), Brier_precip_fc_t3.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Can we shift the distributions to improve things?

#### Compute means of climatological distributions

In [ ]:
mn_fh_precip = precip_fh_m.groupby('time.month').mean('time')
mn_oh_precip = precip_oh_m.groupby('time.month').mean('time')

#### Compute means of forecast distributions (not as function of lead time)

In [ ]:
da_list = []
for init_date in precip_f_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_f_m.sel(init_date=init_date)))
mn_f_precip = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

da_list = []
for init_date in precip_o_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_o_m.sel(init_date=init_date)))
mn_o_precip = xr.concat(da_list, dim='time').groupby('time.month').mean('time')

#### Shift the forecast and observation distributions to have the same means as the historical climatologies

In [ ]:
def shift_mean_per_month(da,shift):
    da_dt = utils.leadtime_to_datetime(da)
    return utils.datetime_to_leadtime(da_dt.groupby('time.month') - shift)

In [ ]:
precip_f_m_ub = precip_f_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_f_precip - mn_fh_precip))
precip_o_m_ub = precip_o_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_o_precip - mn_oh_precip))

#### This process can __introduce some negative rainfall values__

#### Check that distribution means are correct (for forecasts only)

In [ ]:
da_list = []
for init_date in precip_f_m_ub.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_f_m_ub.sel(init_date=init_date)))
test = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

In [ ]:
vmin = -0.001
vmax = 0.001

data = [(mn_fh - test).isel(month=0), (mn_fh - test).isel(month=1), 
        (mn_fh - test).isel(month=2), (mn_fh - test).isel(month=3),
        (mn_fh - test).isel(month=4), (mn_fh - test).isel(month=5),
        (mn_fh - test).isel(month=6), (mn_fh - test).isel(month=7),
        (mn_fh - test).isel(month=8), (mn_fh - test).isel(month=9),
        (mn_fh - test).isel(month=10), (mn_fh - test).isel(month=11)]
text = ['mn 1', 'mn 2', 'mn 3', 'mn 4','mn 5','mn 6','mn 7','mn 8','mn 9','mn 10','mn 11','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Difference between climatological and unbiased forecast means', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.001,0.001,5))

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Now how do the probability anomalies look?

In [ ]:
precip_f_l3_ub = precip_f_m_ub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_fh_precip).mean(dim='ensemble')
precip_f_l1_ub = precip_f_m_ub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_fh_precip).mean(dim='ensemble')

probanom_f_t3_ub = precip_f_l3_ub.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_ub = precip_f_l1_ub.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_ub.isel(lead_time=0), probanom_f_t3_ub.isel(lead_time=3), 
        probanom_f_t3_ub.isel(lead_time=6), probanom_f_t3_ub.isel(lead_time=9),
        probanom_f_t3_ub.isel(lead_time=12), probanom_f_t3_ub.isel(lead_time=15),
        probanom_f_t3_ub.isel(lead_time=18), probanom_f_t3_ub.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Forecast probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_ub.isel(lead_time=0), probanom_f_t1_ub.isel(lead_time=3), 
        probanom_f_t1_ub.isel(lead_time=6), probanom_f_t1_ub.isel(lead_time=9),
        probanom_f_t1_ub.isel(lead_time=12), probanom_f_t1_ub.isel(lead_time=15),
        probanom_f_t1_ub.isel(lead_time=18), probanom_f_t1_ub.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Forecast probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
precip_o_l3_ub = precip_o_m_ub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_oh_precip)
precip_o_l1_ub = precip_o_m_ub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_oh_precip)

probanom_o_t3_ub = precip_o_l3_ub.where(mask).mean(dim='init_date') - 1/3
probanom_o_t1_ub = precip_o_l1_ub.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_o_t3_ub.isel(lead_time=0), probanom_o_t3_ub.isel(lead_time=3), 
        probanom_o_t3_ub.isel(lead_time=6), probanom_o_t3_ub.isel(lead_time=9),
        probanom_o_t3_ub.isel(lead_time=12), probanom_o_t3_ub.isel(lead_time=15),
        probanom_o_t3_ub.isel(lead_time=18), probanom_o_t3_ub.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Observation probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_o_t1_ub.isel(lead_time=0), probanom_o_t1_ub.isel(lead_time=3), 
        probanom_o_t1_ub.isel(lead_time=6), probanom_o_t1_ub.isel(lead_time=9),
        probanom_o_t1_ub.isel(lead_time=12), probanom_o_t1_ub.isel(lead_time=15),
        probanom_o_t1_ub.isel(lead_time=18), probanom_o_t1_ub.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Observation probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Better... but there are still strong trends with lead time
### Are there improvements to the Brier score?

#### Compute Brier score for forecasts

In [ ]:
Brier_precip_f_t3_ub = skill.compute_Brier_score(precip_f_l3_ub, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_f_t1_ub = skill.compute_Brier_score(precip_f_l1_ub, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_f_t1_ub.isel(lead_time=0), Brier_precip_f_t1_ub.isel(lead_time=3), 
        Brier_precip_f_t1_ub.isel(lead_time=7), Brier_precip_f_t1_ub.isel(lead_time=11),
        Brier_precip_f_t3_ub.isel(lead_time=0), Brier_precip_f_t3_ub.isel(lead_time=3),
        Brier_precip_f_t3_ub.isel(lead_time=7), Brier_precip_f_t3_ub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
precip_c_l3_ub = (0 * precip_f_l3_ub + 1/3).where(mask)
precip_c_l1_ub = (0 * precip_f_l1_ub + 1/3).where(mask)

Brier_precip_c_t3_ub = skill.compute_Brier_score(precip_c_l3_ub, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_c_t1_ub = skill.compute_Brier_score(precip_c_l1_ub, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_c_t1_ub.isel(lead_time=0), Brier_precip_c_t1_ub.isel(lead_time=3), 
        Brier_precip_c_t1_ub.isel(lead_time=7), Brier_precip_c_t1_ub.isel(lead_time=11),
        Brier_precip_c_t3_ub.isel(lead_time=0), Brier_precip_c_t3_ub.isel(lead_time=3),
        Brier_precip_c_t3_ub.isel(lead_time=7), Brier_precip_c_t3_ub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_precip_fc_t3_ub = 1 - (Brier_precip_f_t3_ub / Brier_precip_c_t3_ub).where(mask)
Brier_precip_fc_t1_ub = 1 - (Brier_precip_f_t1_ub / Brier_precip_c_t1_ub).where(mask)

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_fc_t1_ub.isel(lead_time=0), Brier_precip_fc_t1_ub.isel(lead_time=3), 
        Brier_precip_fc_t1_ub.isel(lead_time=7), Brier_precip_fc_t1_ub.isel(lead_time=11),
        Brier_precip_fc_t3_ub.isel(lead_time=0), Brier_precip_fc_t3_ub.isel(lead_time=3),
        Brier_precip_fc_t3_ub.isel(lead_time=7), Brier_precip_fc_t3_ub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Slight improvements for tercile 3, deterioration for tercile 1

### Can we shift the distributions as a function of *month and lead time*?

#### Compute means of forecast distributions as function of lead time

In [ ]:
def groupby_lead_and_average(da, over_dims):
    return da.unstack('stacked_init_date_lead_time').groupby('lead_time').mean(over_dims, skipna=True)

In [ ]:
month = (precip_f_m.init_date.dt.month + precip_f_m.lead_time) % 12
month = month.where(month != 0, 12)

precip_f_m.coords['month'] = month
mn_ff_precip = precip_f_m.groupby('month').apply(groupby_lead_and_average, over_dims=['init_date','ensemble'])

precip_o_m.coords['month'] = month
mn_fo_precip = precip_o_m.groupby('month').apply(groupby_lead_and_average, over_dims='init_date')

#### How does the January mean, for example, change as a function of lead time?

In [ ]:
vmin = 0
vmax = 500

data = [mn_ff.isel(month=0,lead_time=0), mn_ff.isel(month=0,lead_time=2), 
        mn_ff.isel(month=0,lead_time=4), mn_ff.isel(month=0,lead_time=6),
        mn_ff.isel(month=0,lead_time=8), mn_ff.isel(month=0,lead_time=10),
        mn_ff.isel(month=0,lead_time=12), mn_ff.isel(month=0,lead_time=14),
        mn_ff.isel(month=0,lead_time=16), mn_ff.isel(month=0,lead_time=18),
        mn_ff.isel(month=0,lead_time=20), mn_ff.isel(month=0,lead_time=22)]
text = ['mn 1','mn 3','mn 5','mn 7','mn 9','mn 11','mn 13','mn 15','mn 17','mn 19','mn 21','mn 23']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=div)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Mean January rainfall', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(0,500,5))

# plt.savefig(saveloc + 'Jan_rainfall_with_lead.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = 0
vmax = 500

data = [mn_ff.isel(month=0,lead_time=0), mn_ff.isel(month=0,lead_time=3), 
        mn_ff.isel(month=0,lead_time=6), mn_ff.isel(month=0,lead_time=9),
        mn_ff.isel(month=0,lead_time=12), mn_ff.isel(month=0,lead_time=15),
        mn_ff.isel(month=0,lead_time=18), mn_ff.isel(month=0,lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=div)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Average January rainfall', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(0,500,5))

plt.savefig(saveloc + 'Jan_rainfall_with_lead.eps', dpi=600, format='eps', bbox_inches='tight')

#### And is the mean across all lead times correct?

In [ ]:
(mn_f.isel(month=0) - mn_ff.isel(month=0).mean('lead_time')).plot();

#### Shift the forecast and observation distributions to have the same means as the historical climatologies
(Still haven't worked out the best way to do this in xarray - currently left with a mostly empty months dimension)

In [ ]:
def unstack_and_shift_mean_per_month(da,shift):
    da_us = da.unstack('stacked_init_date_lead_time')
    the_month = np.ndarray.flatten(da_us.month.values)
    the_month = int(np.unique(the_month[~np.isnan(the_month)]))
    return da_us - shift.sel(month=the_month)

In [ ]:
precip_f_m_fub = precip_f_m.groupby('month').apply(unstack_and_shift_mean_per_month, shift=(mn_ff_precip - mn_fh_precip)) \
                           .mean('month', skipna=True)
precip_o_m_fub = precip_o_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_o_precip - mn_oh_precip))

#### This process can __introduce some negative rainfall values__

#### Check that distribution means are correct (for forecasts only)

In [ ]:
# Lead time units were lost somewhere along the way -----
precip_f_m_fub.lead_time.attrs['units'] = 'MS'

da_list = []
for init_date in precip_f_m_fub.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_f_m_fub.sel(init_date=init_date)))
test = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

In [ ]:
vmin = -0.001
vmax = 0.001

data = [(mn_fh - test).isel(month=0), (mn_fh - test).isel(month=1), 
        (mn_fh - test).isel(month=2), (mn_fh - test).isel(month=3),
        (mn_fh - test).isel(month=4), (mn_fh - test).isel(month=5),
        (mn_fh - test).isel(month=6), (mn_fh - test).isel(month=7),
        (mn_fh - test).isel(month=8), (mn_fh - test).isel(month=9),
        (mn_fh - test).isel(month=10), (mn_fh - test).isel(month=11)]
text = ['mn 1', 'mn 2', 'mn 3', 'mn 4','mn 5','mn 6','mn 7','mn 8','mn 9','mn 10','mn 11','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Difference between climatological and unbiased forecast means', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.001,0.001,5))

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Now how do the probability anomalies look?

In [ ]:
precip_f_l3_fub = precip_f_m_fub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_fh_precip).mean(dim='ensemble')
precip_f_l1_fub = precip_f_m_fub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_fh_precip).mean(dim='ensemble')

probanom_f_t3_fub = precip_f_l3_fub.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_fub = precip_f_l1_fub.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_fub.isel(lead_time=0), probanom_f_t3_fub.isel(lead_time=3), 
        probanom_f_t3_fub.isel(lead_time=6), probanom_f_t3_fub.isel(lead_time=9),
        probanom_f_t3_fub.isel(lead_time=12), probanom_f_t3_fub.isel(lead_time=15),
        probanom_f_t3_fub.isel(lead_time=18), probanom_f_t3_fub.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_fub.isel(lead_time=0), probanom_f_t1_fub.isel(lead_time=3), 
        probanom_f_t1_fub.isel(lead_time=6), probanom_f_t1_fub.isel(lead_time=9),
        probanom_f_t1_fub.isel(lead_time=12), probanom_f_t1_fub.isel(lead_time=15),
        probanom_f_t1_fub.isel(lead_time=18), probanom_f_t1_fub.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Not too bad, though we are getting screwed by negative rainfall values... How do the Brier scores look

#### Compute Brier score for forecasts

In [ ]:
Brier_precip_f_t3_fub = skill.compute_Brier_score(precip_f_l3_fub, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_f_t1_fub = skill.compute_Brier_score(precip_f_l1_fub, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_f_t1_fub.isel(lead_time=0), Brier_precip_f_t1_fub.isel(lead_time=3), 
        Brier_precip_f_t1_fub.isel(lead_time=7), Brier_precip_f_t1_fub.isel(lead_time=11),
        Brier_precip_f_t3_fub.isel(lead_time=0), Brier_precip_f_t3_fub.isel(lead_time=3),
        Brier_precip_f_t3_fub.isel(lead_time=7), Brier_precip_f_t3_fub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
precip_c_l3_fub = (0 * precip_f_l3_fub + 1/3).where(mask)
precip_c_l1_fub = (0 * precip_f_l1_fub + 1/3).where(mask)

Brier_precip_c_t3_fub = skill.compute_Brier_score(precip_c_l3_fub, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_c_t1_fub = skill.compute_Brier_score(precip_c_l1_fub, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_c_t1_fub.isel(lead_time=0), Brier_precip_c_t1_fub.isel(lead_time=3), 
        Brier_precip_c_t1_fub.isel(lead_time=7), Brier_precip_c_t1_fub.isel(lead_time=11),
        Brier_precip_c_t3_fub.isel(lead_time=0), Brier_precip_c_t3_fub.isel(lead_time=3),
        Brier_precip_c_t3_fub.isel(lead_time=7), Brier_precip_c_t3_fub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_precip_fc_t3_fub = 1 - (Brier_precip_f_t3_fub / Brier_precip_c_t3_fub).where(mask)
Brier_precip_fc_t1_fub = 1 - (Brier_precip_f_t1_fub / Brier_precip_c_t1_fub).where(mask)

In [ ]:
vmin= -1
vmax = 1

data = [(Brier_precip_fc_t3_fub).isel(lead_time=0), (Brier_precip_fc_t3_fub).isel(lead_time=1), 
        (Brier_precip_fc_t3_fub).isel(lead_time=3), (Brier_precip_fc_t3_fub).isel(lead_time=7),
        (Brier_precip_fc_t3_fub).isel(lead_time=11), (Brier_precip_fc_t3_fub).isel(lead_time=15),
        (Brier_precip_fc_t3_fub).isel(lead_time=19), (Brier_precip_fc_t3_fub).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Wet tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
#fig.suptitle('Wet tercile');

# plt.savefig(saveloc + 'brier_skill_t_ref3_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -1
vmax = 1

data = [(Brier_precip_fc_t1_fub).isel(lead_time=0), (Brier_precip_fc_t1_fub).isel(lead_time=1), 
        (Brier_precip_fc_t1_fub).isel(lead_time=3), (Brier_precip_fc_t1_fub).isel(lead_time=7),
        (Brier_precip_fc_t1_fub).isel(lead_time=11), (Brier_precip_fc_t1_fub).isel(lead_time=15),
        (Brier_precip_fc_t1_fub).isel(lead_time=19), (Brier_precip_fc_t1_fub).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Dry tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
# fig.suptitle('Dry tercile');

# plt.savefig(saveloc + 'brier_skill_t_ref1_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

### Some improvement...
### Is there a benefit in using the forecast period to define terciles?

#### Calculate tercile bounds

In [ ]:
da_list = []
for init_date in precip_f_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_f_m.sel(init_date=init_date)))
tmp = xr.concat(da_list, dim='time')

t12_f_precip = tmp.groupby('time.month').reduce(np.nanpercentile, dim='time', q=100/3)
t23_f_precip = tmp.groupby('time.month').reduce(np.nanpercentile, dim='time', q=200/3)

#### Tersile forecast probability anomalies

In [ ]:
def below_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') < tercile_bnd)

def above_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') > tercile_bnd)

In [ ]:
precip_f_l3_cp = precip_f_m.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f_precip).mean(dim='ensemble')
precip_f_l1_cp = precip_f_m.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f_precip).mean(dim='ensemble')

probanom_f_t3_precip_cp = precip_f_l3_cp.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_precip_cp = precip_f_l1_cp.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_precip_cp.isel(lead_time=0), probanom_f_t3_precip_cp.isel(lead_time=1), 
        probanom_f_t3_precip_cp.isel(lead_time=3), probanom_f_t3_precip_cp.isel(lead_time=7),
        probanom_f_t3_precip_cp.isel(lead_time=11), probanom_f_t3_precip_cp.isel(lead_time=15),
        probanom_f_t3_precip_cp.isel(lead_time=19), probanom_f_t3_precip_cp.isel(lead_time=23)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Wet tercile : forecast probability anomalies', rotation=0, labelpad=15);

plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
# vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_precip_cp.isel(lead_time=0), probanom_f_t1_precip_cp.isel(lead_time=1), 
        probanom_f_t1_precip_cp.isel(lead_time=3), probanom_f_t1_precip_cp.isel(lead_time=7),
        probanom_f_t1_precip_cp.isel(lead_time=11), probanom_f_t1_precip_cp.isel(lead_time=15),
        probanom_f_t1_precip_cp.isel(lead_time=19), probanom_f_t1_precip_cp.isel(lead_time=23)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Dry tercile : forecast probability anomalies', rotation=0, labelpad=15);

plt.savefig(saveloc + 'probability_anomaly_rainfall_t1.eps', dpi=600, format='eps', bbox_inches='tight')

### Clear trend throughout forecasts
### How do the Brier scores look?

#### Compute Brier score for forecasts

In [ ]:
Brier_precip_f_t3_cp = skill.compute_Brier_score(precip_f_l3_cp, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_f_t1_cp = skill.compute_Brier_score(precip_f_l1_cp, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_f_t3_cp.isel(lead_time=0), Brier_precip_f_t3_cp.isel(lead_time=3), 
        Brier_precip_f_t3_cp.isel(lead_time=7), Brier_precip_f_t3_cp.isel(lead_time=11),
        Brier_precip_f_t1_cp.isel(lead_time=0), Brier_precip_f_t1_cp.isel(lead_time=3),
        Brier_precip_f_t1_cp.isel(lead_time=7), Brier_precip_f_t1_cp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
precip_c_l3_cp = (0 * precip_f_l3_cp + 1/3).where(mask)
precip_c_l1_cp = (0 * precip_f_l1_cp + 1/3).where(mask)

Brier_precip_c_t3_cp = skill.compute_Brier_score(precip_c_l3_cp, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_c_t1_cp = skill.compute_Brier_score(precip_c_l1_cp, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_c_t3_cp.isel(lead_time=0), Brier_precip_c_t3_cp.isel(lead_time=3), 
        Brier_precip_c_t3_cp.isel(lead_time=7), Brier_precip_c_t3_cp.isel(lead_time=11),
        Brier_precip_c_t1_cp.isel(lead_time=0), Brier_precip_c_t1_cp.isel(lead_time=3),
        Brier_precip_c_t1_cp.isel(lead_time=7), Brier_precip_c_t1_cp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_precip_fc_t3_cp = 1 - (Brier_precip_f_t3_cp / Brier_precip_c_t3_cp).where(mask)
Brier_precip_fc_t1_cp = 1 - (Brier_precip_f_t1_cp / Brier_precip_c_t1_cp).where(mask)

In [ ]:
vmin= -1
vmax = 1

data = [(Brier_precip_fc_t3_cp).isel(lead_time=0), (Brier_precip_fc_t3_cp).isel(lead_time=1), 
        (Brier_precip_fc_t3_cp).isel(lead_time=3), (Brier_precip_fc_t3_cp).isel(lead_time=7),
        (Brier_precip_fc_t3_cp).isel(lead_time=11), (Brier_precip_fc_t3_cp).isel(lead_time=15),
        (Brier_precip_fc_t3_cp).isel(lead_time=19), (Brier_precip_fc_t3_cp).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Wet tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
#fig.suptitle('Wet tercile');

plt.savefig(saveloc + 'brier_skill_precip3_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -1
vmax = 1

data = [(Brier_precip_fc_t1_cp).isel(lead_time=0), (Brier_precip_fc_t1_cp).isel(lead_time=1), 
        (Brier_precip_fc_t1_cp).isel(lead_time=3), (Brier_precip_fc_t1_cp).isel(lead_time=7),
        (Brier_precip_fc_t1_cp).isel(lead_time=11), (Brier_precip_fc_t1_cp).isel(lead_time=15),
        (Brier_precip_fc_t1_cp).isel(lead_time=19), (Brier_precip_fc_t1_cp).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Dry tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
# fig.suptitle('Dry tercile');

plt.savefig(saveloc + 'brier_skill_precip1_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
precip_f_terciles_cp = (1 * precip_f_m.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f_precip) + \
                       -1 * precip_f_m.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f_precip)) + 2
precip_f_terciles_cp = precip_f_terciles_cp.where(precip_f_m.notnull())

precip_o_terciles_ub = (1 * precip_o_m_ub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_oh_precip) + \
                       -1 * precip_o_m_ub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_oh_precip)) + 2
precip_o_terciles_ub = precip_o_terciles_ub.where(precip_o_m_ub.notnull())

### Compute rank probability score

#### Forecasts

In [ ]:
RPS_precip_f = skill.compute_rps(precip_f_terciles_cp, precip_o_terciles_ub, bins=[1,2,3], over_dims=None)

#### Climatological mean

In [ ]:
precip_climmean_terciles = xr.concat([0 * precip_o_terciles_ub + 1, 
                                      0 * precip_o_terciles_ub + 2, 
                                      0 * precip_o_terciles_ub + 3], dim='ensemble')

RPS_precip_climmean = skill.compute_rps(precip_climmean_terciles, precip_o_terciles_ub, bins=[1,2,3], over_dims=None)

### Compute rank probability skill score

In [ ]:
RPSS_precip_climmean = 1 - RPS_precip_f / RPS_precip_climmean

In [ ]:
vmin = -1
vmax = 1

data = [(RPSS_precip_climmean).mean('init_date').isel(lead_time=0), (RPSS_precip_climmean).mean('init_date').isel(lead_time=1), 
        (RPSS_precip_climmean).mean('init_date').isel(lead_time=3), (RPSS_precip_climmean).mean('init_date').isel(lead_time=7),
        (RPSS_precip_climmean).mean('init_date').isel(lead_time=11), (RPSS_precip_climmean).mean('init_date').isel(lead_time=15),
        (RPSS_precip_climmean).mean('init_date').isel(lead_time=19), (RPSS_precip_climmean).mean('init_date').isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - RPS_{\mathrm{forecasts}}/RPS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

#plt.savefig(saveloc + 'RPSS_precip1_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

## Break down RPSS according to what ENSO is doing

In [ ]:
def make_Barnston(RPSS, ref_signal, ref_signal_bins=np.linspace(-1.5,1.5,5)):
    leads = np.arange(0,24)
    dat_list = []
    for lead in leads:
        RPSS_lead = RPSS.sel(lead_time=lead)
        ref_lead = ref_signal.sel(lead_time=lead).rename('sst')

        # Generate shifted RPSS -----
        shifts = np.arange(-10,11)
        dat2_list = []
        for shift in shifts:
            dat2_list.append(RPSS_lead.shift(init_date=shift))
        RPSS_shift = xr.concat(dat2_list, dim='shift').rename('shifted')
        RPSS_shift['shift'] = -shifts

        # Bin on reference signal and average RPSS -----
        ref_signal_bin_edges = utils.get_bin_edges(ref_signal_bins)
        RPSS_binned = RPSS_shift.groupby_bins(ref_lead, bins=ref_signal_bin_edges, squeeze=False).mean('init_date') \
                                .rename({'sst_bins':'ref_signal_bins'})
        RPSS_binned['ref_signal_bins'] = ref_signal_bins

        dat_list.append(RPSS_binned)

    RPSS_Barnston = xr.concat(dat_list, dim='lead_time')
    RPSS_Barnston['lead_time'] = leads
    
    return RPSS_Barnston

In [ ]:
RPSS_Barnston = make_Barnston(RPSS_precip_climmean.mean(['lat','lon']), nino34_o)

In [ ]:
vmin = -1
vmax = 1

data = [RPSS_Barnston.isel(lead_time=0), RPSS_Barnston.isel(lead_time=1), 
        RPSS_Barnston.isel(lead_time=3), RPSS_Barnston.isel(lead_time=7),
        RPSS_Barnston.isel(lead_time=11), RPSS_Barnston.isel(lead_time=15),
        RPSS_Barnston.isel(lead_time=19), RPSS_Barnston.isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*4))

count = 1
for idx,dat in enumerate(data):

    ax = plt.subplot(nrow, ncol, count)
    im = ax.contourf(dat['ref_signal_bins'], dat['shift'], dat, np.linspace(-1,1,30), origin='lower',
                     vmin=vmin, vmax=vmax, cmap=seq, extend='both')

    if count % ncol == 0:
        ax.yaxis.tick_right()
    elif (count+ncol-1) % ncol == 0: 
        ax.set_ylabel('Lag [month]')
    else:
        ax.set_yticks([])
    if idx / ncol >= nrow - 1:
        ax.set_xlabel('Nino-3.4')
    ax.set_title(text[count-1], fontsize=17)
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.26)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.020])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - RPS_{\mathrm{forecasts}}/RPS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(vmin,vmax,5))

plt.savefig(saveloc + 'RPSS_precip_nino.eps', dpi=600, format='eps', bbox_inches='tight')

### Compute Brier scores as a function of month

In [ ]:
Brier_precip_f_t3_mn = skill.compute_Brier_score(precip_f_l3_cp, precip_o_l3_ub, over_dims=None)
Brier_precip_f_t1_mn = skill.compute_Brier_score(precip_f_l1_cp, precip_o_l1_ub, over_dims=None)

precip_c_l3_cp = (0 * precip_f_l3_cp + 1/3).where(mask)
precip_c_l1_cp = (0 * precip_f_l1_cp + 1/3).where(mask)

Brier_precip_c_t3_mn = skill.compute_Brier_score(precip_c_l3_cp, precip_o_l3_ub, over_dims=None)
Brier_precip_c_t1_mn = skill.compute_Brier_score(precip_c_l1_cp, precip_o_l1_ub, over_dims=None)

Brier_precip_fc_t3_mn = 1 - (Brier_precip_f_t3_mn / Brier_precip_c_t3_mn).where(mask) \
                            .groupby('init_date.month').mean('init_date')
Brier_precip_fc_t1_mn = 1 - (Brier_precip_f_t1_mn / Brier_precip_c_t1_mn).where(mask) \
                            .groupby('init_date.month').mean('init_date')

In [ ]:
vmin= -1
vmax = 1
months = [1,2,3,4,5,6,7,8,9,10,11,12]

data = [(Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=0), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=1), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=3), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=7),
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=11), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=15),
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=19), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Wet tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('January-February');

# plt.savefig(saveloc + 'brier_skill_precip3_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

### What about if we make our tercile bounds also a function of lead_time?

#### Calculate tercile bounds

In [ ]:
def groupby_lead_and_tercile(da, over_dims, q):
    return da.unstack('stacked_init_date_lead_time').groupby('lead_time') \
             .reduce(np.nanpercentile, dim=over_dims, q=q)

In [ ]:
t12_f_precip = precip_f_m.groupby('month').apply(groupby_lead_and_tercile, 
                                                 over_dims=['init_date','ensemble'], 
                                                 q=100/3)
t23_f_precip = precip_f_m.groupby('month').apply(groupby_lead_and_tercile, 
                                                 over_dims=['init_date','ensemble'], 
                                                 q=200/3)

#### Tersile forecast probability anomalies

In [ ]:
def unstack_and_below_tercile(da,tercile_bnd):
    da_us = da.unstack('stacked_init_date_lead_time')
    the_month = np.ndarray.flatten(da_us.month.values)
    the_month = int(np.unique(the_month[~np.isnan(the_month)]))
    out = da_us < tercile_bnd.sel(month=the_month)
    return out.where(out)

def unstack_and_above_tercile(da,tercile_bnd):
    da_us = da.unstack('stacked_init_date_lead_time')
    the_month = np.ndarray.flatten(da_us.month.values)
    the_month = int(np.unique(the_month[~np.isnan(the_month)]))
    out = da_us > tercile_bnd.sel(month=the_month)
    return out.where(out)

In [ ]:
precip_f_l3_cp = precip_f_m.groupby('month') \
                           .apply(unstack_and_above_tercile, 
                                  tercile_bnd=t23_f_precip) \
                           .mean('month', skipna=True) \
                           .fillna(0).mean(dim='ensemble')
precip_f_l1_cp = precip_f_m.groupby('month') \
                           .apply(unstack_and_below_tercile, 
                                  tercile_bnd=t12_f_precip) \
                           .mean('month', skipna=True) \
                           .fillna(0).mean(dim='ensemble')

probanom_f_t3_precip_cp = precip_f_l3_cp.mean(dim='init_date').where(mask) - 1/3
probanom_f_t1_precip_cp = precip_f_l1_cp.mean(dim='init_date').where(mask) - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_precip_cp.isel(lead_time=0), probanom_f_t3_precip_cp.isel(lead_time=1), 
        probanom_f_t3_precip_cp.isel(lead_time=3), probanom_f_t3_precip_cp.isel(lead_time=7),
        probanom_f_t3_precip_cp.isel(lead_time=11), probanom_f_t3_precip_cp.isel(lead_time=15),
        probanom_f_t3_precip_cp.isel(lead_time=19), probanom_f_t3_precip_cp.isel(lead_time=23)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Wet tercile : forecast probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
# vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_precip_cp.isel(lead_time=0), probanom_f_t1_precip_cp.isel(lead_time=1), 
        probanom_f_t1_precip_cp.isel(lead_time=3), probanom_f_t1_precip_cp.isel(lead_time=7),
        probanom_f_t1_precip_cp.isel(lead_time=11), probanom_f_t1_precip_cp.isel(lead_time=15),
        probanom_f_t1_precip_cp.isel(lead_time=19), probanom_f_t1_precip_cp.isel(lead_time=23)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Dry tercile : forecast probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t1.eps', dpi=600, format='eps', bbox_inches='tight')

### Compute Brier scores as a function of month

In [ ]:
Brier_precip_f_t3_mn = skill.compute_Brier_score(precip_f_l3_cp, precip_o_l3_ub, over_dims=None)
Brier_precip_f_t1_mn = skill.compute_Brier_score(precip_f_l1_cp, precip_o_l1_ub, over_dims=None)

precip_c_l3_cp = (0 * precip_f_l3_cp + 1/3).where(mask)
precip_c_l1_cp = (0 * precip_f_l1_cp + 1/3).where(mask)

Brier_precip_c_t3_mn = skill.compute_Brier_score(precip_c_l3_cp, precip_o_l3_ub, over_dims=None)
Brier_precip_c_t1_mn = skill.compute_Brier_score(precip_c_l1_cp, precip_o_l1_ub, over_dims=None)

Brier_precip_fc_t3_mn = 1 - (Brier_precip_f_t3_mn / Brier_precip_c_t3_mn).where(mask) \
                            .groupby('init_date.month').mean('init_date')
Brier_precip_fc_t1_mn = 1 - (Brier_precip_f_t1_mn / Brier_precip_c_t1_mn).where(mask) \
                            .groupby('init_date.month').mean('init_date')

In [ ]:
vmin= -1
vmax = 1
months = [1,2,3,4,5,6,7,8,9,10,11,12]

data = [(Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=0), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=1), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=3), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=7),
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=11), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=15),
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=19), 
        (Brier_precip_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Wet tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('Full year');

# plt.savefig(saveloc + 'brier_skill_precip3_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

### Determine the mean and standard deviations of percentage correct for different lead times

In [ ]:
in_t3_f = precip_f_m_fub_cp.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f_precip)
in_t3_o = precip_o_m_ub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_oh_precip)

perc_correct = (in_t3_f == in_t3_o).where(mask).mean('ensemble')

In [ ]:
perc_correct.mean(['lat','lon']).isel(lead_time=0).std()

In [ ]:
perc_correct.mean(['lat','lon']).isel(lead_time=2).std()

### Build the contingency tables and Heidke scores

In [ ]:
# Project tersile boundaries over all dimensions
proj_monthly = lambda shape, proj: utils.datetime_to_leadtime(utils.leadtime_to_datetime(shape).groupby('time.month')+proj)

t12_f_precip_proj = (0*precip_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t12_f_precip)
t23_f_precip_proj = (0*precip_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t23_f_precip)
t12_o_precip_proj = (0*precip_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t12_oh_precip)
t23_o_precip_proj = (0*precip_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t23_oh_precip)

In [ ]:
contingency = skill.compute_contingency_table(precip_f_m, precip_o_m, 
                                              [-np.inf, t12_f_precip_proj, t23_f_precip_proj, np.inf],
                                              [-np.inf, t12_o_precip_proj, t23_o_precip_proj, np.inf],
                                              over_dims=['init_date','ensemble','lat','lon'])

In [ ]:
(contingency / contingency.sum(['comparison_category','reference_category'])).sel(lead_time=12)

In [ ]:
Hiedke = skill.compute_Heidke_score(contingency)

In [ ]:
fig = plt.figure(figsize=(9, 5))

plt.plot([-5, 30], [0, 0], '-', color=(0.8,0.8,0.8))
plt.plot(Hiedke.lead_time+1,Hiedke.mean(['lat','lon']), color=(0.1, 0.55, 0.75), linewidth=2)
plt.xlabel('Lead time [months]')
plt.ylabel('Heidke skill score')
plt.xlim(0,25)
plt.ylim(-0.1,0.1)
plt.savefig(saveloc + 'Heidke_skill_precip_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

### Can we shift the distributions as a function of *month and lead time*?

#### Compute means of current current forecast distributions

In [ ]:
mn_f_precip_cp = tmp.groupby('time.month').mean(['time','ensemble'])

#### Shift the forecast and observation distributions to have the same means as the current climatologies
(Still haven't worked out the best way to do this in xarray - currently left with a mostly empty months dimension)

In [ ]:
precip_f_m_fub_cp = precip_f_m.groupby('month').apply(unstack_and_shift_mean_per_month, shift=(mn_ff_precip - mn_f_precip_cp)) \
                              .mean('month', skipna=True)

#### Check that distribution means are correct (for forecasts only)

In [ ]:
# Lead time units were lost somewhere along the way -----
precip_f_m_fub_cp.lead_time.attrs['units'] = 'MS'

da_list = []
for init_date in precip_f_m_fub_cp.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_f_m_fub_cp.sel(init_date=init_date)))
test = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

In [ ]:
vmin = -0.001
vmax = 0.001

data = [(mn_f_precip_cp - test).isel(month=0), (mn_f_precip_cp - test).isel(month=1), 
        (mn_f_precip_cp - test).isel(month=2), (mn_f_precip_cp - test).isel(month=3),
        (mn_f_precip_cp - test).isel(month=4), (mn_f_precip_cp - test).isel(month=5),
        (mn_f_precip_cp - test).isel(month=6), (mn_f_precip_cp - test).isel(month=7),
        (mn_f_precip_cp - test).isel(month=8), (mn_f_precip_cp - test).isel(month=9),
        (mn_f_precip_cp - test).isel(month=10), (mn_f_precip_cp - test).isel(month=11)]
text = ['mn 1', 'mn 2', 'mn 3', 'mn 4','mn 5','mn 6','mn 7','mn 8','mn 9','mn 10','mn 11','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Difference between climatological and unbiased forecast means', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.001,0.001,5))

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Now how do the probability anomalies look?

In [ ]:
precip_f_l3_fub_cp = precip_f_m_fub_cp.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f_precip).mean(dim='ensemble')
precip_f_l1_fub_cp = precip_f_m_fub_cp.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f_precip).mean(dim='ensemble')

probanom_f_t3_fub_precip_cp = precip_f_l3_fub_cp.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_fub_precip_cp = precip_f_l1_fub_cp.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_fub_precip_cp.isel(lead_time=0), probanom_f_t3_fub_precip_cp.isel(lead_time=3), 
        probanom_f_t3_fub_precip_cp.isel(lead_time=6), probanom_f_t3_fub_precip_cp.isel(lead_time=9),
        probanom_f_t3_fub_precip_cp.isel(lead_time=12), probanom_f_t3_fub_precip_cp.isel(lead_time=15),
        probanom_f_t3_fub_precip_cp.isel(lead_time=18), probanom_f_t3_fub_precip_cp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_fub_precip_cp.isel(lead_time=0), probanom_f_t1_fub_precip_cp.isel(lead_time=3), 
        probanom_f_t1_fub_precip_cp.isel(lead_time=6), probanom_f_t1_fub_precip_cp.isel(lead_time=9),
        probanom_f_t1_fub_precip_cp.isel(lead_time=12), probanom_f_t1_fub_precip_cp.isel(lead_time=15),
        probanom_f_t1_fub_precip_cp.isel(lead_time=18), probanom_f_t1_fub_precip_cp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Not too bad... How do the Brier scores look

#### Compute Brier score for forecasts

In [ ]:
Brier_precip_f_t3_fub_cp = skill.compute_Brier_score(precip_f_l3_fub_cp, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_f_t1_fub_cp = skill.compute_Brier_score(precip_f_l1_fub_cp, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_f_t3_fub_cp.isel(lead_time=0), Brier_precip_f_t3_fub_cp.isel(lead_time=3), 
        Brier_precip_f_t3_fub_cp.isel(lead_time=7), Brier_precip_f_t3_fub_cp.isel(lead_time=11),
        Brier_precip_f_t1_fub_cp.isel(lead_time=0), Brier_precip_f_t1_fub_cp.isel(lead_time=3),
        Brier_precip_f_t1_fub_cp.isel(lead_time=7), Brier_precip_f_t1_fub_cp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
precip_c_l3_fub_cp = (0 * precip_f_l3_fub_cp + 1/3).where(mask)
precip_c_l1_fub_cp = (0 * precip_f_l1_fub_cp + 1/3).where(mask)

Brier_precip_c_t3_fub_cp = skill.compute_Brier_score(precip_c_l3_fub_cp, precip_o_l3_ub, over_dims=['init_date'])
Brier_precip_c_t1_fub_cp = skill.compute_Brier_score(precip_c_l1_fub_cp, precip_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_c_t3_fub_cp.isel(lead_time=0), Brier_precip_c_t3_fub_cp.isel(lead_time=3), 
        Brier_precip_c_t3_fub_cp.isel(lead_time=7), Brier_precip_c_t3_fub_cp.isel(lead_time=11),
        Brier_precip_c_t1_fub_cp.isel(lead_time=0), Brier_precip_c_t1_fub_cp.isel(lead_time=3),
        Brier_precip_c_t1_fub_cp.isel(lead_time=7), Brier_precip_c_t1_fub_cp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_precip_fc_t3_fub_cp = 1 - (Brier_precip_f_t3_fub_cp / Brier_precip_c_t3_fub_cp).where(mask)
Brier_precip_fc_t1_fub_cp = 1 - (Brier_precip_f_t1_fub_cp / Brier_precip_c_t1_fub_cp).where(mask)

In [ ]:
vmin= -1
vmax = 1

data = [(Brier_precip_fc_t3_fub_cp).isel(lead_time=0), (Brier_precip_fc_t3_fub_cp).isel(lead_time=1), 
        (Brier_precip_fc_t3_fub_cp).isel(lead_time=3), (Brier_precip_fc_t3_fub_cp).isel(lead_time=7),
        (Brier_precip_fc_t3_fub_cp).isel(lead_time=11), (Brier_precip_fc_t3_fub_cp).isel(lead_time=15),
        (Brier_precip_fc_t3_fub_cp).isel(lead_time=19), (Brier_precip_fc_t3_fub_cp).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Wet tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
#fig.suptitle('Wet tercile');

# plt.savefig(saveloc + 'brier_skill_t_ref3_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -1
vmax = 1

data = [(Brier_precip_fc_t1_fub_cp).isel(lead_time=0), (Brier_precip_fc_t1_fub_cp).isel(lead_time=1), 
        (Brier_precip_fc_t1_fub_cp).isel(lead_time=3), (Brier_precip_fc_t1_fub_cp).isel(lead_time=7),
        (Brier_precip_fc_t1_fub_cp).isel(lead_time=11), (Brier_precip_fc_t1_fub_cp).isel(lead_time=15),
        (Brier_precip_fc_t1_fub_cp).isel(lead_time=19), (Brier_precip_fc_t1_fub_cp).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Dry tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
# fig.suptitle('Dry tercile');

# plt.savefig(saveloc + 'brier_skill_t_ref1_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

# Local temperature metrics

In [ ]:
folder = '/OSM/CBR/OA_DCFP/data/intermediate_products/squ027/'
file = 'cafe.fcst_2yr.v1.t_ref.bundle.daily.aus.native_grid.20030101_20151201.nc'

temp_f_d = xr.open_dataset(folder + file)['h500']- 273.15
temp_f_d['init_date'] = temp_f_d.init_date.astype('<M8[D]')                  

In [ ]:
folder = '//OSM/CBR/OA_DCFP/data/observations/jra55/isobaric/011_tmp/cat/'
file = 'jra.55.tmp.000.1958010100_2016123118.nc'

temp_o_raw = xr.open_dataset(folder + file)['TMP_GDS0_HTGL'] \
               .rename({'g0_lon_2':'lon','g0_lat_1':'lat','initial_time0_hours':'time'}) \
               .sel(time=slice('2002-01-01','2016-12-31')) \
               .interp(lat=temp_f_d.lat, lon=temp_f_d.lon, method='linear') - 273.15
temp_o_raw['time'] = temp_o_raw.time.astype('<M8[D]')
temp_o_d = utils.stack_by_init_date(temp_o_raw, temp_f_d.init_date.values, 2*365)

In [ ]:
landsea = xr.open_dataset('/OSM/CBR/OA_DCFP/data/intermediate_products/masks/ncar_landsea_mask.nc')['LSMASK']
mask = landsea.interp(lat=temp_f_d.lat, lon=temp_f_d.lon, method='nearest')

In [ ]:
# Compute monthly temperature -----
convert_to_monthly = lambda data : utils.datetime_to_leadtime(
                                   utils.leadtime_to_datetime(data).resample(time='MS').mean(dim='time',skipna=False))

temp_f_m = temp_f_d.groupby('init_date').apply(convert_to_monthly).where(mask)
temp_o_m = temp_o_d.groupby('init_date').apply(convert_to_monthly).where(mask)

### What about if we make our tercile bounds also a function of lead_time?

#### Calculate tercile bounds

In [ ]:
def groupby_lead_and_tercile(da, over_dims, q):
    return da.unstack('stacked_init_date_lead_time').groupby('lead_time') \
             .reduce(np.nanpercentile, dim=over_dims, q=q)

In [ ]:
t12_f_temp = temp_f_m.groupby('month').apply(groupby_lead_and_tercile, 
                                                 over_dims=['init_date','ensemble'], 
                                                 q=100/3)
t23_f_temp = temp_f_m.groupby('month').apply(groupby_lead_and_tercile, 
                                                 over_dims=['init_date','ensemble'], 
                                                 q=200/3)

#### Tersile forecast probability anomalies

In [ ]:
def unstack_and_below_tercile(da,tercile_bnd):
    da_us = da.unstack('stacked_init_date_lead_time')
    the_month = np.ndarray.flatten(da_us.month.values)
    the_month = int(np.unique(the_month[~np.isnan(the_month)]))
    out = da_us < tercile_bnd.sel(month=the_month)
    return out.where(out)

def unstack_and_above_tercile(da,tercile_bnd):
    da_us = da.unstack('stacked_init_date_lead_time')
    the_month = np.ndarray.flatten(da_us.month.values)
    the_month = int(np.unique(the_month[~np.isnan(the_month)]))
    out = da_us > tercile_bnd.sel(month=the_month)
    return out.where(out)

In [ ]:
temp_f_l3_cp = temp_f_m.groupby('month') \
                           .apply(unstack_and_above_tercile, 
                                  tercile_bnd=t23_f_temp) \
                           .mean('month', skipna=True) \
                           .fillna(0).mean(dim='ensemble')
temp_f_l1_cp = temp_f_m.groupby('month') \
                           .apply(unstack_and_below_tercile, 
                                  tercile_bnd=t12_f_temp) \
                           .mean('month', skipna=True) \
                           .fillna(0).mean(dim='ensemble')

probanom_f_t3_temp_cp = temp_f_l3_cp.mean(dim='init_date').where(mask) - 1/3
probanom_f_t1_temp_cp = temp_f_l1_cp.mean(dim='init_date').where(mask) - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_temp_cp.isel(lead_time=0), probanom_f_t3_temp_cp.isel(lead_time=1), 
        probanom_f_t3_temp_cp.isel(lead_time=3), probanom_f_t3_temp_cp.isel(lead_time=7),
        probanom_f_t3_temp_cp.isel(lead_time=11), probanom_f_t3_temp_cp.isel(lead_time=15),
        probanom_f_t3_temp_cp.isel(lead_time=19), probanom_f_t3_temp_cp.isel(lead_time=23)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Hot tercile : forecast probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
# vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_temp_cp.isel(lead_time=0), probanom_f_t1_temp_cp.isel(lead_time=1), 
        probanom_f_t1_temp_cp.isel(lead_time=3), probanom_f_t1_temp_cp.isel(lead_time=7),
        probanom_f_t1_temp_cp.isel(lead_time=11), probanom_f_t1_temp_cp.isel(lead_time=15),
        probanom_f_t1_temp_cp.isel(lead_time=19), probanom_f_t1_temp_cp.isel(lead_time=23)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Cold tercile : forecast probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t1.eps', dpi=600, format='eps', bbox_inches='tight')

### Compute Brier scores as a function of month

In [ ]:
Brier_temp_f_t3_mn = skill.compute_Brier_score(temp_f_l3_cp, temp_o_l3_ub, over_dims=None)
Brier_temp_f_t1_mn = skill.compute_Brier_score(temp_f_l1_cp, temp_o_l1_ub, over_dims=None)

temp_c_l3_cp = (0 * temp_f_l3_cp + 1/3).where(mask)
temp_c_l1_cp = (0 * temp_f_l1_cp + 1/3).where(mask)

Brier_temp_c_t3_mn = skill.compute_Brier_score(temp_c_l3_cp, temp_o_l3_ub, over_dims=None)
Brier_temp_c_t1_mn = skill.compute_Brier_score(temp_c_l1_cp, temp_o_l1_ub, over_dims=None)

Brier_temp_fc_t3_mn = 1 - (Brier_temp_f_t3_mn / Brier_temp_c_t3_mn).where(mask) \
                            .groupby('init_date.month').mean('init_date')
Brier_temp_fc_t1_mn = 1 - (Brier_temp_f_t1_mn / Brier_temp_c_t1_mn).where(mask) \
                            .groupby('init_date.month').mean('init_date')

In [ ]:
vmin= -1
vmax = 1
months = [1,2,3,4,5,6]

data = [(Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=0), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=1), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=3), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=7),
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=11), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=15),
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=19), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Hot tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('First half of year');

# plt.savefig(saveloc + 'brier_skill_precip3_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin= -1
vmax = 1
months = [7,8,9,10,11,12]

data = [(Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=0), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=1), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=3), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=7),
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=11), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=15),
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=19), 
        (Brier_temp_fc_t3_mn).sel(month=months).mean('month').isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Cold tercile : $1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('Second half of year');

# plt.savefig(saveloc + 'brier_skill_precip3_cclim.eps', dpi=600, format='eps', bbox_inches='tight')

### Compute tersiles edges from historical data

In [ ]:
def cftime_to_datetime64(time,shift_year=0):
    """ 
    Convert cftime object to datetime64 object
    
    Assumes times are sequential
    """

    if (time.values[0].timetuple()[0]+shift_year < 1678) | (time.values[-1].timetuple()[0]+shift_year > 2261):
        raise ValueError('Cannot create datetime64 object for years outside on 1678-2262')
        
    return np.array([np.datetime64(time.values[i].replace(year=time.values[i].timetuple()[0]+shift_year) \
                                                 .strftime(), 'ns') \
                                                 for i in range(len(time))])

#### Control run

In [ ]:
folder = '/OSM/CBR/OA_DCFP/data2/model_output/CAFE/controls/c2/OUTPUT/'
file = 'atmos_daily_04*.nc'

temp_fh_d = xr.open_mfdataset(folder + file, concat_dim='time', parallel=True)['t_ref'] \
                .sel(lat=slice(-47,-10), lon=slice(110,155)) \
                .where(mask).compute() - 273.15
temp_fh_d['time'] = cftime_to_datetime64(temp_fh_d['time'],1300)
temp_fh_d = temp_fh_d.squeeze()

# Keep only last 40 years -----
temp_fh_d = temp_fh_d.sel(time=slice('1760-01-01','1799-12-31'))

temp_fh_m = temp_fh_d.resample(time='MS').mean('time')

t12_fh_temp = temp_fh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=100/3)
t23_fh_temp = temp_fh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=200/3)

#### JRA

In [ ]:
folder = '//OSM/CBR/OA_DCFP/data/observations/jra55/isobaric/011_tmp/cat/'
file = 'jra.55.tmp.000.1958010100_2016123118.nc'

temp_oh_d = xr.open_dataset(folder + file)['TMP_GDS0_HTGL'] \
                .sel(initial_time0_hours=slice('1958-01-01','1997-12-31')) \
                .rename({'g0_lon_2':'lon','g0_lat_1':'lat', 'initial_time0_hours':'time'}) \
                .interp(lat=temp_fh_d.lat, lon=temp_fh_d.lon, method='linear') \
                .where(mask) - 273.15
temp_oh_d = temp_oh_d.squeeze()

temp_oh_m = temp_oh_d.resample(time='MS').mean('time')

t12_oh_temp = temp_oh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=100/3)
t23_oh_temp = temp_oh_m.groupby('time.month').reduce(np.nanpercentile, dim='time', q=200/3)

### Probability anomalies

In [ ]:
def below_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') < tercile_bnd)

def above_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') > tercile_bnd)

#### Tersile forecast probability anomalies

In [ ]:
temp_f_l3 = temp_f_m.groupby('init_date').apply(above_tercile, tercile_bnd=t23_fh_temp).mean(dim='ensemble')
temp_f_l1 = temp_f_m.groupby('init_date').apply(below_tercile, tercile_bnd=t12_fh_temp).mean(dim='ensemble')

probanom_f_t3_temp = temp_f_l3.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_temp = temp_f_l1.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_temp.isel(lead_time=0), probanom_f_t3_temp.isel(lead_time=3), 
        probanom_f_t3_temp.isel(lead_time=7), probanom_f_t3_temp.isel(lead_time=11),
        probanom_f_t1_temp.isel(lead_time=1), probanom_f_t1_temp.isel(lead_time=4),
        probanom_f_t1_temp.isel(lead_time=7), probanom_f_t1_temp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Forecast probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

#### Tersile observation probability anomalies

In [ ]:
temp_o_l3 = temp_o_m.groupby('init_date').apply(above_tercile, tercile_bnd=t23_oh_temp)
temp_o_l1 = temp_o_m.groupby('init_date').apply(below_tercile, tercile_bnd=t12_oh_temp)

probanom_o_t3_temp = temp_o_l3.where(mask).mean(dim='init_date') - 1/3
probanom_o_t1_temp = temp_o_l1.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_o_t3_temp.isel(lead_time=0), probanom_o_t3_temp.isel(lead_time=3), 
        probanom_o_t3_temp.isel(lead_time=7), probanom_o_t3_temp.isel(lead_time=11),
        probanom_o_t1_temp.isel(lead_time=1), probanom_o_t1_temp.isel(lead_time=4),
        probanom_o_t1_temp.isel(lead_time=7), probanom_o_t1_temp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Observation probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Clearly the distributions have drifted, so Brier skill is poor

#### Compute Brier score for forecasts

In [ ]:
Brier_temp_f_t3 = skill.compute_Brier_score(temp_f_l3, temp_o_l3, over_dims=['init_date'])
Brier_temp_f_t1 = skill.compute_Brier_score(temp_f_l1, temp_o_l1, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_f_t1.isel(lead_time=0), Brier_temp_f_t1.isel(lead_time=3), 
        Brier_temp_f_t1.isel(lead_time=7), Brier_temp_f_t1.isel(lead_time=11),
        Brier_temp_f_t3.isel(lead_time=0), Brier_temp_f_t3.isel(lead_time=3),
        Brier_temp_f_t3.isel(lead_time=7), Brier_temp_f_t3.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
temp_c_l3 = (0 * temp_f_l3 + 1/3).where(mask)
temp_c_l1 = (0 * temp_f_l1 + 1/3).where(mask)

Brier_temp_c_t3 = skill.compute_Brier_score(temp_c_l3, temp_o_l3, over_dims=['init_date'])
Brier_temp_c_t1 = skill.compute_Brier_score(temp_c_l1, temp_o_l1, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_c_t1.isel(lead_time=0), Brier_temp_c_t1.isel(lead_time=3), 
        Brier_temp_c_t1.isel(lead_time=7), Brier_temp_c_t1.isel(lead_time=11),
        Brier_temp_c_t3.isel(lead_time=0), Brier_temp_c_t3.isel(lead_time=3),
        Brier_temp_c_t3.isel(lead_time=7), Brier_temp_c_t3.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_temp_fc_t3 = 1 - (Brier_temp_f_t3 / Brier_temp_c_t3).where(mask)
Brier_temp_fc_t1 = 1 - (Brier_temp_f_t1 / Brier_temp_c_t1).where(mask)

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_fc_t1.isel(lead_time=0), Brier_temp_fc_t1.isel(lead_time=3), 
        Brier_temp_fc_t1.isel(lead_time=7), Brier_temp_fc_t1.isel(lead_time=11),
        Brier_temp_fc_t3.isel(lead_time=0), Brier_temp_fc_t3.isel(lead_time=3),
        Brier_temp_fc_t3.isel(lead_time=7), Brier_temp_fc_t3.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Can we shift the distributions to improve things?

#### Compute means of climatological distributions

In [ ]:
mn_fh_temp = temp_fh_m.groupby('time.month').mean('time')
mn_oh_temp = temp_oh_m.groupby('time.month').mean('time')

#### Compute means of forecast distributions (not as function of lead time)

In [ ]:
da_list = []
for init_date in temp_f_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(temp_f_m.sel(init_date=init_date)))
mn_f_temp = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

da_list = []
for init_date in temp_o_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(temp_o_m.sel(init_date=init_date)))
mn_o_temp = xr.concat(da_list, dim='time').groupby('time.month').mean('time')

#### Shift the forecast and observation distributions to have the same means as the historical climatologies

In [ ]:
def shift_mean_per_month(da,shift):
    da_dt = utils.leadtime_to_datetime(da)
    return utils.datetime_to_leadtime(da_dt.groupby('time.month') - shift)

In [ ]:
temp_f_m_ub = temp_f_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_f_temp - mn_fh_temp))
temp_o_m_ub = temp_o_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_o_temp - mn_oh_temp))

#### Check that distribution means are correct (for forecasts only)

In [ ]:
da_list = []
for init_date in temp_f_m_ub.init_date.values:
    da_list.append(utils.leadtime_to_datetime(temp_f_m_ub.sel(init_date=init_date)))
test = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

In [ ]:
vmin = -0.001
vmax = 0.001

data = [(mn_fh_temp - test).isel(month=0), (mn_fh_temp - test).isel(month=1), 
        (mn_fh_temp - test).isel(month=2), (mn_fh_temp - test).isel(month=3),
        (mn_fh_temp - test).isel(month=4), (mn_fh_temp - test).isel(month=5),
        (mn_fh_temp - test).isel(month=6), (mn_fh_temp - test).isel(month=7),
        (mn_fh_temp - test).isel(month=8), (mn_fh_temp - test).isel(month=9),
        (mn_fh_temp - test).isel(month=10), (mn_fh_temp - test).isel(month=11)]
text = ['mn 1', 'mn 2', 'mn 3', 'mn 4','mn 5','mn 6','mn 7','mn 8','mn 9','mn 10','mn 11','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Difference between climatological and unbiased forecast means', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.001,0.001,5))

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Now how do the probability anomalies look?

In [ ]:
temp_f_l3_ub = temp_f_m_ub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_fh_temp).mean(dim='ensemble')
temp_f_l1_ub = temp_f_m_ub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_fh_temp).mean(dim='ensemble')

probanom_f_t3_ub_temp = temp_f_l3_ub.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_ub_temp = temp_f_l1_ub.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_ub_temp.isel(lead_time=0), probanom_f_t3_ub_temp.isel(lead_time=3), 
        probanom_f_t3_ub_temp.isel(lead_time=6), probanom_f_t3_ub_temp.isel(lead_time=9),
        probanom_f_t3_ub_temp.isel(lead_time=12), probanom_f_t3_ub_temp.isel(lead_time=15),
        probanom_f_t3_ub_temp.isel(lead_time=18), probanom_f_t3_ub_temp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Forecast probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_ub_temp.isel(lead_time=0), probanom_f_t1_ub_temp.isel(lead_time=3), 
        probanom_f_t1_ub_temp.isel(lead_time=6), probanom_f_t1_ub_temp.isel(lead_time=9),
        probanom_f_t1_ub_temp.isel(lead_time=12), probanom_f_t1_ub_temp.isel(lead_time=15),
        probanom_f_t1_ub_temp.isel(lead_time=18), probanom_f_t1_ub_temp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Forecast probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
temp_o_l3_ub = temp_o_m_ub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_oh_temp)
temp_o_l1_ub = temp_o_m_ub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_oh_temp)

probanom_o_t3_ub_temp = temp_o_l3_ub.where(mask).mean(dim='init_date') - 1/3
probanom_o_t1_ub_temp = temp_o_l1_ub.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_o_t3_ub_temp.isel(lead_time=0), probanom_o_t3_ub_temp.isel(lead_time=3), 
        probanom_o_t3_ub_temp.isel(lead_time=6), probanom_o_t3_ub_temp.isel(lead_time=9),
        probanom_o_t3_ub_temp.isel(lead_time=12), probanom_o_t3_ub_temp.isel(lead_time=15),
        probanom_o_t3_ub_temp.isel(lead_time=18), probanom_o_t3_ub_temp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Observation probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_o_t1_ub_temp.isel(lead_time=0), probanom_o_t1_ub_temp.isel(lead_time=3), 
        probanom_o_t1_ub_temp.isel(lead_time=6), probanom_o_t1_ub_temp.isel(lead_time=9),
        probanom_o_t1_ub_temp.isel(lead_time=12), probanom_o_t1_ub_temp.isel(lead_time=15),
        probanom_o_t1_ub_temp.isel(lead_time=18), probanom_o_t1_ub_temp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Observation probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Better... but there are still strong trends with lead time
### Are there improvements to the Brier score?

#### Compute Brier score for forecasts

In [ ]:
Brier_temp_f_t3_ub = skill.compute_Brier_score(temp_f_l3_ub, temp_o_l3_ub, over_dims=['init_date'])
Brier_temp_f_t1_ub = skill.compute_Brier_score(temp_f_l1_ub, temp_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_f_t1_ub.isel(lead_time=0), Brier_temp_f_t1_ub.isel(lead_time=3), 
        Brier_temp_f_t1_ub.isel(lead_time=7), Brier_temp_f_t1_ub.isel(lead_time=11),
        Brier_temp_f_t3_ub.isel(lead_time=0), Brier_temp_f_t3_ub.isel(lead_time=3),
        Brier_temp_f_t3_ub.isel(lead_time=7), Brier_temp_f_t3_ub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
temp_c_l3_ub = (0 * temp_f_l3_ub + 1/3).where(mask)
temp_c_l1_ub = (0 * temp_f_l1_ub + 1/3).where(mask)

Brier_temp_c_t3_ub = skill.compute_Brier_score(temp_c_l3_ub, temp_o_l3_ub, over_dims=['init_date'])
Brier_temp_c_t1_ub = skill.compute_Brier_score(temp_c_l1_ub, temp_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_c_t1_ub.isel(lead_time=0), Brier_temp_c_t1_ub.isel(lead_time=3), 
        Brier_temp_c_t1_ub.isel(lead_time=7), Brier_temp_c_t1_ub.isel(lead_time=11),
        Brier_temp_c_t3_ub.isel(lead_time=0), Brier_temp_c_t3_ub.isel(lead_time=3),
        Brier_temp_c_t3_ub.isel(lead_time=7), Brier_temp_c_t3_ub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_temp_fc_t3_ub = 1 - (Brier_temp_f_t3_ub / Brier_temp_c_t3_ub).where(mask)
Brier_temp_fc_t1_ub = 1 - (Brier_temp_f_t1_ub / Brier_temp_c_t1_ub).where(mask)

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_fc_t1_ub.isel(lead_time=0), Brier_temp_fc_t1_ub.isel(lead_time=3), 
        Brier_temp_fc_t1_ub.isel(lead_time=7), Brier_temp_fc_t1_ub.isel(lead_time=11),
        Brier_temp_fc_t3_ub.isel(lead_time=0), Brier_temp_fc_t3_ub.isel(lead_time=3),
        Brier_temp_fc_t3_ub.isel(lead_time=7), Brier_temp_fc_t3_ub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Improvements for tercile 1 and tercile 3

### Can we shift the distributions as a function of *month and lead time*?

#### Compute means of forecast distributions as function of lead time

In [ ]:
def groupby_lead_and_average(da, over_dims):
    return da.unstack('stacked_init_date_lead_time').groupby('lead_time').mean(over_dims, skipna=True)

In [ ]:
month = (temp_f_m.init_date.dt.month + temp_f_m.lead_time) % 12
month = month.where(month != 0, 12)

temp_f_m.coords['month'] = month
mn_ff_temp = temp_f_m.groupby('month').apply(groupby_lead_and_average, over_dims=['init_date','ensemble'])

temp_o_m.coords['month'] = month
mn_fo_temp = temp_o_m.groupby('month').apply(groupby_lead_and_average, over_dims='init_date')

#### How does the January mean, for example, change as a function of lead time?

In [ ]:
vmin = 0
vmax = 36

data = [mn_ff_temp.isel(month=0,lead_time=0), mn_ff_temp.isel(month=0,lead_time=2), 
        mn_ff_temp.isel(month=0,lead_time=4), mn_ff_temp.isel(month=0,lead_time=6),
        mn_ff_temp.isel(month=0,lead_time=8), mn_ff_temp.isel(month=0,lead_time=10),
        mn_ff_temp.isel(month=0,lead_time=12), mn_ff_temp.isel(month=0,lead_time=14),
        mn_ff_temp.isel(month=0,lead_time=16), mn_ff_temp.isel(month=0,lead_time=18),
        mn_ff_temp.isel(month=0,lead_time=20), mn_ff_temp.isel(month=0,lead_time=22)]
text = ['mn 1','mn 3','mn 5','mn 7','mn 9','mn 11','mn 13','mn 15','mn 17','mn 19','mn 21','mn 23']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))
plt.set_cmap('YlGnBu_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Mean January temperature', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(0,36,5))

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### It doesn't really...

#### And is the mean across all lead times correct?

In [ ]:
(mn_f_temp.isel(month=0) - mn_ff_temp.isel(month=0).mean('lead_time')).plot();

#### Shift the forecast and observation distributions to have the same means as the historical climatologies
(Still haven't worked out the best way to do this in xarray - currently left with a mostly empty months dimension)

In [ ]:
def unstack_and_shift_mean_per_month(da,shift):
    da_us = da.unstack('stacked_init_date_lead_time')
    the_month = np.ndarray.flatten(da_us.month.values)
    the_month = int(np.unique(the_month[~np.isnan(the_month)]))
    return da_us - shift.sel(month=the_month)

In [ ]:
temp_f_m_fub = temp_f_m.groupby('month').apply(unstack_and_shift_mean_per_month, shift=(mn_ff_temp - mn_fh_temp)) \
                           .mean('month', skipna=True)
temp_o_m_fub = temp_o_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_o_temp - mn_oh_temp))

#### Check that distribution means are correct (for forecasts only)

In [ ]:
# Lead time units were lost somewhere along the way -----
temp_f_m_fub.lead_time.attrs['units'] = 'MS'

da_list = []
for init_date in temp_f_m_fub.init_date.values:
    da_list.append(utils.leadtime_to_datetime(temp_f_m_fub.sel(init_date=init_date)))
test = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

In [ ]:
vmin = -0.001
vmax = 0.001

data = [(mn_fh_temp - test).isel(month=0), (mn_fh_temp - test).isel(month=1), 
        (mn_fh_temp - test).isel(month=2), (mn_fh_temp - test).isel(month=3),
        (mn_fh_temp - test).isel(month=4), (mn_fh_temp - test).isel(month=5),
        (mn_fh_temp - test).isel(month=6), (mn_fh_temp - test).isel(month=7),
        (mn_fh_temp - test).isel(month=8), (mn_fh_temp - test).isel(month=9),
        (mn_fh_temp - test).isel(month=10), (mn_fh_temp - test).isel(month=11)]
text = ['mn 1', 'mn 2', 'mn 3', 'mn 4','mn 5','mn 6','mn 7','mn 8','mn 9','mn 10','mn 11','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Difference between climatological and unbiased forecast means', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.001,0.001,5))

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Now how do the probability anomalies look?

In [ ]:
temp_f_l3_fub = temp_f_m_fub.groupby('init_date').apply(above_tercile, tercile_bnd=t23_fh_temp).mean(dim='ensemble')
temp_f_l1_fub = temp_f_m_fub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_fh_temp).mean(dim='ensemble')

probanom_f_t3_fub_temp = temp_f_l3_fub.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_fub_temp = temp_f_l1_fub.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_fub_temp.isel(lead_time=0), probanom_f_t3_fub_temp.isel(lead_time=3), 
        probanom_f_t3_fub_temp.isel(lead_time=6), probanom_f_t3_fub_temp.isel(lead_time=9),
        probanom_f_t3_fub_temp.isel(lead_time=12), probanom_f_t3_fub_temp.isel(lead_time=15),
        probanom_f_t3_fub_temp.isel(lead_time=18), probanom_f_t3_fub_temp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_fub_temp.isel(lead_time=0), probanom_f_t1_fub_temp.isel(lead_time=3), 
        probanom_f_t1_fub_temp.isel(lead_time=6), probanom_f_t1_fub_temp.isel(lead_time=9),
        probanom_f_t1_fub_temp.isel(lead_time=12), probanom_f_t1_fub_temp.isel(lead_time=15),
        probanom_f_t1_fub_temp.isel(lead_time=18), probanom_f_t1_fub_temp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Not too bad... How do the Brier scores look

#### Compute Brier score for forecasts

In [ ]:
Brier_temp_f_t3_fub = skill.compute_Brier_score(temp_f_l3_fub, temp_o_l3_ub, over_dims=['init_date'])
Brier_temp_f_t1_fub = skill.compute_Brier_score(temp_f_l1_fub, temp_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_f_t3_fub.isel(lead_time=0), Brier_temp_f_t3_fub.isel(lead_time=3), 
        Brier_temp_f_t3_fub.isel(lead_time=7), Brier_temp_f_t3_fub.isel(lead_time=11),
        Brier_temp_f_t1_fub.isel(lead_time=0), Brier_temp_f_t1_fub.isel(lead_time=3),
        Brier_temp_f_t1_fub.isel(lead_time=7), Brier_temp_f_t1_fub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
temp_c_l3_fub = (0 * temp_f_l3_fub + 1/3).where(mask)
temp_c_l1_fub = (0 * temp_f_l1_fub + 1/3).where(mask)

Brier_temp_c_t3_fub = skill.compute_Brier_score(temp_c_l3_fub, temp_o_l3_ub, over_dims=['init_date'])
Brier_temp_c_t1_fub = skill.compute_Brier_score(temp_c_l1_fub, temp_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_c_t3_fub.isel(lead_time=0), Brier_temp_c_t3_fub.isel(lead_time=3), 
        Brier_temp_c_t3_fub.isel(lead_time=7), Brier_temp_c_t3_fub.isel(lead_time=11),
        Brier_temp_c_t1_fub.isel(lead_time=0), Brier_temp_c_t1_fub.isel(lead_time=3),
        Brier_temp_c_t1_fub.isel(lead_time=7), Brier_temp_c_t1_fub.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_temp_fc_t3_fub = 1 - (Brier_temp_f_t3_fub / Brier_temp_c_t3_fub).where(mask)
Brier_temp_fc_t1_fub = 1 - (Brier_temp_f_t1_fub / Brier_temp_c_t1_fub).where(mask)

In [ ]:
vmin= -1
vmax = 1

data = [(Brier_temp_fc_t3_fub).isel(lead_time=0), (Brier_temp_fc_t3_fub).isel(lead_time=1), 
        (Brier_temp_fc_t3_fub).isel(lead_time=3), (Brier_temp_fc_t3_fub).isel(lead_time=7),
        (Brier_temp_fc_t3_fub).isel(lead_time=11), (Brier_temp_fc_t3_fub).isel(lead_time=15),
        (Brier_temp_fc_t3_fub).isel(lead_time=19), (Brier_temp_fc_t3_fub).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('Warm tercile');

plt.savefig(saveloc + 'brier_skill_t_ref3_hclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -1
vmax = 1

data = [(Brier_temp_fc_t1_fub).isel(lead_time=0), (Brier_temp_fc_t1_fub).isel(lead_time=1), 
        (Brier_temp_fc_t1_fub).isel(lead_time=3), (Brier_temp_fc_t1_fub).isel(lead_time=7),
        (Brier_temp_fc_t1_fub).isel(lead_time=11), (Brier_temp_fc_t1_fub).isel(lead_time=15),
        (Brier_temp_fc_t1_fub).isel(lead_time=19), (Brier_temp_fc_t1_fub).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('Cool tercile');

plt.savefig(saveloc + 'brier_skill_t_ref1_hclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

### Not much improvement...
### Is there a benefit in using the forecast period to define terciles?

#### Calculate tercile bounds

In [ ]:
da_list = []
for init_date in temp_f_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(temp_f_m.sel(init_date=init_date)))
tmp = xr.concat(da_list, dim='time')

t12_f_temp = tmp.groupby('time.month').reduce(np.nanpercentile, dim='time', q=100/3)
t23_f_temp = tmp.groupby('time.month').reduce(np.nanpercentile, dim='time', q=200/3)

#### Tersile forecast probability anomalies

In [ ]:
def below_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') < tercile_bnd)

def above_tercile(fcst,tercile_bnd):
    return utils.datetime_to_leadtime(
               utils.leadtime_to_datetime(fcst).groupby('time.month') > tercile_bnd)

In [ ]:
temp_f_l3_cp = temp_f_m.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f_temp).mean(dim='ensemble')
temp_f_l1_cp = temp_f_m.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f_temp).mean(dim='ensemble')

probanom_f_t3_temp_cp = temp_f_l3_cp.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_temp_cp = temp_f_l1_cp.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_temp_cp.isel(lead_time=0), probanom_f_t3_temp_cp.isel(lead_time=3), 
        probanom_f_t3_temp_cp.isel(lead_time=7), probanom_f_t3_temp_cp.isel(lead_time=11),
        probanom_f_t1_temp_cp.isel(lead_time=1), probanom_f_t1_temp_cp.isel(lead_time=4),
        probanom_f_t1_temp_cp.isel(lead_time=7), probanom_f_t1_temp_cp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Forecast probability anomalies', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Clear trend throughout forecasts
### Can we shift the distributions as a function of *month and lead time*?

#### Compute means of current current forecast distributions

In [ ]:
mn_f_temp_cp = tmp.groupby('time.month').mean(['time','ensemble'])

#### Shift the forecast and observation distributions to have the same means as the current climatologies
(Still haven't worked out the best way to do this in xarray - currently left with a mostly empty months dimension)

In [ ]:
temp_f_m_fub_cp = temp_f_m.groupby('month').apply(unstack_and_shift_mean_per_month, shift=(mn_ff_temp - mn_f_temp_cp)) \
                           .mean('month', skipna=True)

#### Check that distribution means are correct (for forecasts only)

In [ ]:
# Lead time units were lost somewhere along the way -----
temp_f_m_fub_cp.lead_time.attrs['units'] = 'MS'

da_list = []
for init_date in temp_f_m_fub_cp.init_date.values:
    da_list.append(utils.leadtime_to_datetime(temp_f_m_fub_cp.sel(init_date=init_date)))
test = xr.concat(da_list, dim='time').groupby('time.month').mean(['time','ensemble'])

In [ ]:
vmin = -0.001
vmax = 0.001

data = [(mn_f_temp_cp - test).isel(month=0), (mn_f_temp_cp - test).isel(month=1), 
        (mn_f_temp_cp - test).isel(month=2), (mn_f_temp_cp - test).isel(month=3),
        (mn_f_temp_cp - test).isel(month=4), (mn_f_temp_cp - test).isel(month=5),
        (mn_f_temp_cp - test).isel(month=6), (mn_f_temp_cp - test).isel(month=7),
        (mn_f_temp_cp - test).isel(month=8), (mn_f_temp_cp - test).isel(month=9),
        (mn_f_temp_cp - test).isel(month=10), (mn_f_temp_cp - test).isel(month=11)]
text = ['mn 1', 'mn 2', 'mn 3', 'mn 4','mn 5','mn 6','mn 7','mn 8','mn 9','mn 10','mn 11','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 6; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.20)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.040])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Difference between climatological and unbiased forecast means', rotation=0, labelpad=15);
cbar.set_ticks(np.linspace(-0.001,0.001,5))

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

### Now how do the probability anomalies look?

In [ ]:
temp_f_l3_fub_cp = temp_f_m_fub_cp.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f_temp).mean(dim='ensemble')
temp_f_l1_fub_cp = temp_f_m_fub_cp.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f_temp).mean(dim='ensemble')

probanom_f_t3_fub_temp_cp = temp_f_l3_fub_cp.where(mask).mean(dim='init_date') - 1/3
probanom_f_t1_fub_temp_cp = temp_f_l1_fub_cp.where(mask).mean(dim='init_date') - 1/3

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t3_fub_temp_cp.isel(lead_time=0), probanom_f_t3_fub_temp_cp.isel(lead_time=3), 
        probanom_f_t3_fub_temp_cp.isel(lead_time=6), probanom_f_t3_fub_temp_cp.isel(lead_time=9),
        probanom_f_t3_fub_temp_cp.isel(lead_time=12), probanom_f_t3_fub_temp_cp.isel(lead_time=15),
        probanom_f_t3_fub_temp_cp.isel(lead_time=18), probanom_f_t3_fub_temp_cp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 3', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -0.3
vmax = 0.3

data = [probanom_f_t1_fub_temp_cp.isel(lead_time=0), probanom_f_t1_fub_temp_cp.isel(lead_time=3), 
        probanom_f_t1_fub_temp_cp.isel(lead_time=6), probanom_f_t1_fub_temp_cp.isel(lead_time=9),
        probanom_f_t1_fub_temp_cp.isel(lead_time=12), probanom_f_t1_fub_temp_cp.isel(lead_time=15),
        probanom_f_t1_fub_temp_cp.isel(lead_time=18), probanom_f_t1_fub_temp_cp.isel(lead_time=21)]
text = ['mn 1', 'mn 4', 'mn 7', 'mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('Unbiased forecast probability anomalies, tercile 1', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'probability_anomaly_rainfall_t3.eps', dpi=600, format='eps', bbox_inches='tight')

### Not too bad... How do the Brier scores look

#### Compute Brier score for forecasts

In [ ]:
Brier_temp_f_t3_fub_cp = skill.compute_Brier_score(temp_f_l3_fub_cp, temp_o_l3_ub, over_dims=['init_date'])
Brier_temp_f_t1_fub_cp = skill.compute_Brier_score(temp_f_l1_fub_cp, temp_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_f_t3_fub_cp.isel(lead_time=0), Brier_temp_f_t3_fub_cp.isel(lead_time=3), 
        Brier_temp_f_t3_fub_cp.isel(lead_time=7), Brier_temp_f_t3_fub_cp.isel(lead_time=11),
        Brier_temp_f_t1_fub_cp.isel(lead_time=0), Brier_temp_f_t1_fub_cp.isel(lead_time=3),
        Brier_temp_f_t1_fub_cp.isel(lead_time=7), Brier_temp_f_t1_fub_cp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{forecasts}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute the Brier score for climatology

In [ ]:
temp_c_l3_fub_cp = (0 * temp_f_l3_fub_cp + 1/3).where(mask)
temp_c_l1_fub_cp = (0 * temp_f_l1_fub_cp + 1/3).where(mask)

Brier_temp_c_t3_fub_cp = skill.compute_Brier_score(temp_c_l3_fub_cp, temp_o_l3_ub, over_dims=['init_date'])
Brier_temp_c_t1_fub_cp = skill.compute_Brier_score(temp_c_l1_fub_cp, temp_o_l1_ub, over_dims=['init_date'])

In [ ]:
vmin = -1
vmax = 1

data = [Brier_temp_c_t3_fub_cp.isel(lead_time=0), Brier_temp_c_t3_fub_cp.isel(lead_time=3), 
        Brier_temp_c_t3_fub_cp.isel(lead_time=7), Brier_temp_c_t3_fub_cp.isel(lead_time=11),
        Brier_temp_c_t1_fub_cp.isel(lead_time=0), Brier_temp_c_t1_fub_cp.isel(lead_time=3),
        Brier_temp_c_t1_fub_cp.isel(lead_time=7), Brier_temp_c_t1_fub_cp.isel(lead_time=11)]
text = ['mn 1', 'mn 4', 'mn 8', 'mn 12','mn 1','mn 4','mn 8','mn 12']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);

# plt.savefig(saveloc + 'brier_skill_chance_rainfall.eps', dpi=600, format='eps', bbox_inches='tight')

#### Compute relative skill

In [ ]:
Brier_temp_fc_t3_fub_cp = 1 - (Brier_temp_f_t3_fub_cp / Brier_temp_c_t3_fub_cp).where(mask)
Brier_temp_fc_t1_fub_cp = 1 - (Brier_temp_f_t1_fub_cp / Brier_temp_c_t1_fub_cp).where(mask)

In [ ]:
vmin= -1
vmax = 1

data = [(Brier_temp_fc_t3_fub_cp).isel(lead_time=0), (Brier_temp_fc_t3_fub_cp).isel(lead_time=1), 
        (Brier_temp_fc_t3_fub_cp).isel(lead_time=3), (Brier_temp_fc_t3_fub_cp).isel(lead_time=7),
        (Brier_temp_fc_t3_fub_cp).isel(lead_time=11), (Brier_temp_fc_t3_fub_cp).isel(lead_time=15),
        (Brier_temp_fc_t3_fub_cp).isel(lead_time=19), (Brier_temp_fc_t3_fub_cp).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('Warm tercile');

plt.savefig(saveloc + 'brier_skill_t_ref3_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

In [ ]:
vmin = -1
vmax = 1

data = [(Brier_temp_fc_t1_fub_cp).isel(lead_time=0), (Brier_temp_fc_t1_fub_cp).isel(lead_time=1), 
        (Brier_temp_fc_t1_fub_cp).isel(lead_time=3), (Brier_temp_fc_t1_fub_cp).isel(lead_time=7),
        (Brier_temp_fc_t1_fub_cp).isel(lead_time=11), (Brier_temp_fc_t1_fub_cp).isel(lead_time=15),
        (Brier_temp_fc_t1_fub_cp).isel(lead_time=19), (Brier_temp_fc_t1_fub_cp).isel(lead_time=23)]
text = ['mn 1', 'mn 2', 'mn 4', 'mn 8','mn 12','mn 16','mn 20','mn 24']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('Cool tercile');

plt.savefig(saveloc + 'brier_skill_t_ref1_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

### Compute contingency table

In [ ]:
# Project tersile boundaries over all dimensions
proj_monthly = lambda shape, proj: utils.datetime_to_leadtime(utils.leadtime_to_datetime(shape).groupby('time.month')+proj)

t12_f_temp_proj = (0*temp_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t12_f_temp)
t23_f_temp_proj = (0*temp_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t23_f_temp)
t12_o_temp_proj = (0*temp_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t12_oh_temp)
t23_o_temp_proj = (0*temp_f_m_fub_cp).groupby('init_date').apply(proj_monthly, proj=t23_oh_temp)

In [ ]:
contingency = skill.compute_contingency_table(temp_f_m_fub_cp, temp_o_m_fub, 
                                              [-np.inf, t12_f_temp_proj, t23_f_temp_proj, np.inf],
                                              [-np.inf, t12_o_temp_proj, t23_o_temp_proj, np.inf],
                                              over_dims=['init_date','ensemble'])

In [ ]:
Hiedke = skill.compute_Heidke_score(contingency)

In [ ]:
fig = plt.figure(figsize=(9, 5))

plt.plot([-5, 30], [0, 0], '-', color=(0.8,0.8,0.8))
plt.plot(Hiedke.lead_time+1,Hiedke.mean(['lat','lon']))
plt.xlabel('Lead time [months]')
plt.ylabel('Heidke skill score')
plt.xlim(0,25)
plt.ylim(0,1)
# plt.savefig(saveloc + 'Heidke_skill_t_ref_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

### The contingency table suggests that lead time 1 is better than subsequent lead times, whereas the Brier scores show the opposite. This is perhaps an indication that skill is not evenly spread across each experiment (initial date). Can we see this?
### Let's compute the Brier score at month 1 using subsets of experiments

In [ ]:
use_last = [20, 40, 60, 80, 100, 120, 140, 156]
dat_list = []
for n in use_last:
    keep_f = temp_f_l1_fub_cp.isel(init_date=range(len(temp_f_l1_fub_cp.init_date)-n,len(temp_f_l1_fub_cp.init_date)))
    keep_o = temp_o_l1_ub.isel(init_date=range(len(temp_o_l1_ub.init_date)-n,len(temp_o_l1_ub.init_date)))
    tmp = 1 - (skill.compute_Brier_score(keep_f, keep_o, over_dims=['init_date']) / Brier_temp_c_t1_fub_cp).where(mask)
    tmp['n'] = n
    dat_list.append(tmp.isel(lead_time=0))

Brier_subsets = xr.concat(dat_list, dim='n')

In [ ]:
vmin = -1
vmax = 1

data = [Brier_subsets.sel(n=20), Brier_subsets.sel(n=40), 
        Brier_subsets.sel(n=60), Brier_subsets.sel(n=80), 
        Brier_subsets.sel(n=100), Brier_subsets.sel(n=120), 
        Brier_subsets.sel(n=140), Brier_subsets.sel(n=156)]
text = ['n = 20', 'n = 40', 'n = 60', 'n = 80','n = 100','n = 120','n = 140','n = 156']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*3))
plt.set_cmap('RdBu')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.028])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS_{\mathrm{forecasts}}/BS_{\mathrm{climatology}}$', rotation=0, labelpad=15);
fig.suptitle('Cool tercile');

# plt.savefig(saveloc + 'brier_skill_t_ref1_cclim_debiased.eps', dpi=600, format='eps', bbox_inches='tight')

### Compute the proportion of the ensemble that has the correct result

In [ ]:
def compute_proportion_correct(cmp_likelihood, ref_logical, over_dims, cmp_prob=None):
    if over_dims is None:
        over_dims = []   
    N = (0*cmp_likelihood + 1).sum(dim=over_dims, skipna=True)

    ref_binary = ref_logical.copy()*1
    
    # Calculate total Brier score -----
    Brier = (1 / N) * ((cmp_likelihood - ref_binary) ** 2).sum(dim=over_dims, skipna=True) \
                                                          .rename('Brier_score')

In [ ]:
in_t1_f = temp_f_m_fub_cp.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f_temp)
in_t1_o = temp_o_m_ub.groupby('init_date').apply(below_tercile, tercile_bnd=t12_oh_temp)

perc_correct = (in_t1_f == in_t1_o).where(mask).mean('ensemble')

In [ ]:
perc_correct.mean(['lat','lon']).isel(lead_time=0).std()

In [ ]:
perc_correct.mean(['lat','lon']).isel(lead_time=1).std()

In [ ]:
perc_correct.mean(['lat','lon']).isel(lead_time=1).plot()

In [ ]:
Brier_temp_f_t1_fub_cp = skill.compute_Brier_score(temp_f_l1_fub_cp, temp_o_l1_ub, over_dims=None)

In [ ]:
Brier_temp_f_t1_fub_cp.mean(['lat','lon']).isel(lead_time=0).plot()
Brier_temp_f_t1_fub_cp.mean(['lat','lon']).isel(lead_time=1).plot()

In [ ]:
Brier_temp_f_t1_fub_cp.mean(['lat','lon']).isel(lead_time=1).plot()

#### Create months-leads array to group by

In [ ]:
da_list = []
for init_date in precip_f_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_f_m.sel(init_date=init_date)))
precip_f_m_stacked = xr.concat(da_list, dim='time')

da_list = []
for init_date in precip_o_m.init_date.values:
    da_list.append(utils.leadtime_to_datetime(precip_o_m.sel(init_date=init_date)))
precip_o_m_stacked = xr.concat(da_list, dim='time')

months = np.array([str(i).zfill(2) + '-' for i in precip_f_m_stacked.time.dt.month.values])
leads = np.array([str(i).zfill(2)  for i in np.tile(precip_f_m.lead_time,len(precip_f_m.init_date))])
month_leads = precip_f_m_stacked['time'].copy()
month_leads.values = np.core.defchararray.add(months, leads)

#### Determine forecast means for each month-lead

In [ ]:
mn_ffh = precip_f_m_stacked.groupby(month_leads).mean(['time','ensemble'])
mn_foh = precip_o_m_stacked.groupby(month_leads).mean('time')

#### Shift the forecast and observation distributions to have the same means as the historical climatologies

In [ ]:
def shift_mean_per_month(da,shift):
    da_dt = utils.leadtime_to_datetime(da)
    return utils.datetime_to_leadtime(da_dt.groupby('time.month') - shift)

In [ ]:
precip_f_m_ub = precip_f_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_f - mn_fh))
precip_o_m_ub = precip_o_m.groupby('init_date').apply(shift_mean_per_month, shift=(mn_o - mn_oh))

In [ ]:
month = xr.DataArray(np.tile(np.linspace(1,12,12),len(precip_f_m.lead_time)),coords=[mn_ffh.time], dims=['time'])
lead = xr.DataArray(np.tile(np.linspace(0,23,24),12),coords=[mn_ffh.time], dims=['time'])
mn_ffh['month'] = month
mn_ffh['lead'] = lead

In [ ]:
mn_ffh.time.values

In [ ]:
mn_ffh.set_index(time = ['month', 'lead'])

In [ ]:
def shift_mean_per_monthlead(da,shift):
    leads = da.lead_time
    da_dt = utils.leadtime_to_datetime(da)
    return utils.datetime_to_leadtime(da_dt.groupby('time.month') - shift)

In [ ]:
precip_f_m_fub = precip_f_m.groupby('init_date').apply(shift_mean_per_monthlead, shift=(mn_f - mn_fh))
precip_o_m_fub = precip_o_m.groupby('init_date').apply(shift_mean_per_monthlead, shift=(mn_o - mn_oh))

In [ ]:
da = list(precip_f_m.groupby('init_date'))[0][1]

In [ ]:
leads = da.lead_time
da_dt = utils.leadtime_to_datetime(da)

In [ ]:
utils.datetime_to_leadtime(da_dt.groupby('time.month') - (mn_f - mn_fh))

### Compute the Brier score relative to chance

#### Compute Brier score for forecasts

In [ ]:
precip_f_l1 = precip_f.groupby('init_date').apply(below_tercile, tercile_bnd=t12_f).mean(dim='ensemble')
precip_o_l1 = precip_o.groupby('init_date').apply(below_tercile, tercile_bnd=t12_o)

precip_f_l3 = precip_f.groupby('init_date').apply(above_tercile, tercile_bnd=t23_f).mean(dim='ensemble')
precip_o_l3 = precip_o.groupby('init_date').apply(above_tercile, tercile_bnd=t23_o)

# Compute the Brier score for forecasts -----
Brier_precip_f_t1 = skill.compute_Brier_score(precip_f_l1, precip_o_l1, over_dims=['init_date'])
Brier_precip_f_t3 = skill.compute_Brier_score(precip_f_l3, precip_o_l3, over_dims=['init_date'])

### Compute the Brier score relative to climatology

In [ ]:
vmin = -1
vmax = 1

data = [Brier_precip_fx_t1.isel(lead_time=3), Brier_precip_fx_t3.isel(lead_time=3), 
        Brier_precip_fc_t1.isel(lead_time=3), Brier_precip_fc_t3.isel(lead_time=3),]
text = ['(a)','(b)','(c)','(d)']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 17})

ncol = 2; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(8, nrow*4.1))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):

    trans = cartopy.crs.PlateCarree()
    ax = plt.subplot(nrow, ncol, count, projection=cartopy.crs.PlateCarree(central_longitude=180))
    ax.coastlines(color='black')
    # ax.add_feature(cartopy.feature.LAND)
    extent = [dat.lon.min(), dat.lon.max(), 
              dat.lat.min(), dat.lat.max()]
    
    im = ax.imshow(dat, origin='lower', extent=extent, transform=trans, vmin=vmin, vmax=vmax, cmap=seq)
    # im = ax.contourf(dat.lon, dat.lat, dat, np.linspace(-1,1,30), origin='lower', transform=trans, 
    #                   vmin=vmin, vmax=vmax, cmap=seq, extend='both')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=3, colors='w')
    # im = ax.contour(dat.lon, dat.lat, dat, [-0.9, -0.6, -0.3, 0.3, 0.6, 0.9], origin='lower', transform=trans, 
    #                  vmin=vmin, vmax=vmax, linewidths=1, colors='k')
    
    ax.set_xlim(-69,-24)
    ax.set_ylim(-46,-9)
    ax.text(-67.5, -44, text[idx])
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
    gl.xlines = False
    gl.ylines = False
    gl.xlabels_top = False
    if count % ncol == 0:
        gl.ylabels_left = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    else:
        gl.ylabels_right = False
        gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
    gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    count += 1
    
plt.tight_layout()
fig.subplots_adjust(bottom=0.16)
cbar_ax = fig.add_axes([0.15, 0.13, 0.7, 0.02])
fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both');
cbar_ax.set_xlabel('$1 - BS / BS_{\mathrm{ref}}$', rotation=0, labelpad=15);
plt.savefig(saveloc + 'brier_skill_chanceclim_rainfall_month4.eps', dpi=600, format='eps', bbox_inches='tight')

### Focus on Tasmania

In [ ]:
# Make regions mask -----
tas = mask.copy()

T = [[14,1],[14,2]]
for point in T:
    tas[point[1],point[0]] = 2
    
tas = (tas - 1).where((tas - 1) > 0, 0)

In [ ]:
tas.where(tas, drop=True)

In [ ]:
# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica Neue') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 12})

fig = plt.figure(figsize=(9, 4.4))
plt.set_cmap('Spectral_r')

trans = cartopy.crs.PlateCarree()
ax = plt.axes(projection=cartopy.crs.PlateCarree(central_longitude=180))
ax.coastlines(color='black')
ax.add_feature(cartopy.feature.LAND)

ax.plot(146.25-180, -43.483146, 'r+', linewidth=2, markersize=15)
lx = (143.75+146.25)/2-180
rx = (146.25+148.75)/2-180
by = (-45.505618+-43.483146)/2
ty = (-43.483146+-41.460674)/2
ax.plot([lx, rx, rx, lx, lx], [by, by, ty, ty, by], 'r--')
by = ty
ax.plot(146.25-180, -41.460674, 'r+', linewidth=2, markersize=15)
ty = (-41.460674+-39.438202)/2
ax.plot([lx, lx, rx, rx], [by, ty, ty, by], 'r--')

ax.set_xlim(-36,-31)
ax.set_ylim(-45,-40)
ax.stock_img()
gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True)
gl.xlines = False
gl.ylines = False
gl.xlabels_top = False
if count % ncol == 0:
    gl.ylabels_left = False
    gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
else:
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator([-90, 0, 90, 180])
gl.ylocator = mticker.FixedLocator([-90, -60, 0, 60, 90])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
  
plt.savefig(saveloc + 'tas_gridboxes.eps', dpi=600, format='eps', bbox_inches='tight')

#### Plot timeseries

In [ ]:
col = (0.1, 0.55, 0.75)
col_f = ((0.1+3)/4, (0.55+3)/4, (0.75+3)/4)

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 16})

fig = plt.figure(figsize=(11, 3.5))

data = [utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2011-02-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2011-08-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2012-02-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2012-08-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2013-02-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2013-08-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2014-02-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2014-08-01')),
        utils.leadtime_to_datetime(precip_f.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2015-02-01')),]

for idx,dat in enumerate(data):
    plt.plot(dat['time'],dat.transpose(),color=col_f);

for idx,dat in enumerate(data):
    plt.plot(dat['time'],dat.mean('ensemble'),'-',color=col,linewidth=2);
    plt.plot(dat['time'][0],dat.mean('ensemble')[0],'o',color=col);
    
data = [utils.leadtime_to_datetime(precip_o.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2011-02-01')),
        utils.leadtime_to_datetime(precip_o.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2013-02-01')),
        utils.leadtime_to_datetime(precip_o.where(tas).mean(['lat','lon'], skipna=True).sel(init_date = '2015-02-01')),]

for idx,dat in enumerate(data):
    plt.plot(dat['time'],dat,'k--',linewidth=2)

plt.ylabel('Monthly rainfall [mm]')
plt.xlabel('Time')
plt.xlim('2011','2017');

plt.savefig(saveloc + 'tas_rainfall_timeseries.eps', dpi=600, format='eps', bbox_inches='tight')

#### Plot rank histograms

In [ ]:
rank = skill.compute_rank_histogram(precip_f_m.where(tas), precip_o_m.where(tas), 
                                    over_dims=['init_date','lat','lon'])

In [ ]:
col = (0.1, 0.55, 0.75)

data = [rank.isel(lead_time=0), rank.isel(lead_time=3), 
        rank.isel(lead_time=6), rank.isel(lead_time=9),
        rank.isel(lead_time=12), rank.isel(lead_time=15),
        rank.isel(lead_time=18), rank.isel(lead_time=21)]
text = ['mn 1','mn 4','mn 7','mn 10','mn 13','mn 16','mn 19','mn 22']

# Generate plots -----
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 16})

ncol = 4; nrow = int(np.ceil(len(data)/ncol));

fig = plt.figure(figsize=(11, nrow*2.1))
plt.set_cmap('Spectral_r')

count = 1
for idx,dat in enumerate(data):
    
    ax = plt.subplot(nrow, ncol, count)
    ax.bar(dat.bins,dat,color=col,edgecolor='k')
    
    ax.set_yticks(np.arange(0,1,0.1))
    ax.set_yticklabels([0.0,0.1,0.2,0.3,0.4])
    ax.set_xticks(np.arange(0,13,1))
    ax.set_xticklabels([0,'',2,'',4,'',6,'',8,'',10,'',12])
    
    if idx % ncol == 0:
        ax.set_ylabel('Likelihood')
    else:
        ax.set_yticklabels([])
        
    if idx / ncol >= nrow - 1:
        ax.set_xlabel('Rank')
    else:
        ax.set_xticklabels([])

    ax.set_ylim(0, 0.4)
    ax.text(7.6,0.33,text[idx])
    
    count += 1
    
plt.savefig(saveloc + 'tas_rainfall_rankhist.eps', dpi=600, format='eps', bbox_inches='tight')

#### Plot contingency table

In [ ]:
def compute_contingency_table(da_cmp, da_ref, category_edges_cmp, category_edges_ref, over_dims):
    """ Return contingency table for given categories """
    
    if over_dims is None:
        over_dims = []
    
    cmp_edges = [(category_edges_cmp[i], category_edges_cmp[i+1],i+1) for i in range(len(category_edges_cmp)-1)]
    ref_edges = [(category_edges_ref[i], category_edges_ref[i+1],i+1) for i in range(len(category_edges_ref)-1)]
    da_list = []
    for category in itertools.product(cmp_edges, ref_edges):
        da_temp = (((da_cmp >= category[0][0]) & (da_cmp < category[0][1])) & \
                   ((da_ref >= category[1][0]) & (da_ref < category[1][1]))).sum(dim=over_dims)
        da_temp.coords['comparison_category'] = category[0][2]
        da_temp.coords['reference_category'] = category[1][2]
        da_list.append(da_temp)
    
    if len(da_list) == 1:
        return da_list[0]
    else:
        return xr.concat(da_list, dim='stack').set_index(stack=['comparison_category', 'reference_category']).unstack('stack')      

In [ ]:
# Project tersile boundaries over all dimensions
proj_monthly = lambda shape, proj: utils.datetime_to_leadtime(utils.leadtime_to_datetime(shape).groupby('time.month')+proj)

t12_f_proj = (0*precip_f).groupby('init_date').apply(proj_monthly, proj=t12_f)
t23_f_proj = (0*precip_f).groupby('init_date').apply(proj_monthly, proj=t23_f)
t12_o_proj = (0*precip_f).groupby('init_date').apply(proj_monthly, proj=t12_o)
t23_o_proj = (0*precip_f).groupby('init_date').apply(proj_monthly, proj=t23_o)

In [ ]:
contingency = compute_contingency_table(precip_f, precip_o, 
                                        [-np.inf, t12_f_proj, t23_f_proj, np.inf],
                                        [-np.inf, t12_o_proj, t23_o_proj, np.inf],
                                        over_dims=['init_date','ensemble']).where(tas).sum(['lat','lon'])

In [ ]:
(contingency / contingency.sum(['comparison_category','reference_category'])).sel(lead_time=12)